# Library

In [206]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

## [로드] 말소 데이터

In [207]:
ex_col_df = pd.read_excel('data/220819_22.06월기준_말소_컬럼.xlsx')
ex_col_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   컬럼명     42 non-null     object
dtypes: object(1)
memory usage: 464.0+ bytes


In [208]:
ex_col_df.head()

,컬럼명
0,차량관리번호
1,차량등록번호
2,본거지법정동코드
3,소유자구분
4,차종_등록정보


In [209]:
ex_col = ex_col_df['컬럼명']
len(ex_col)

42

In [210]:
ex_col

0          차량관리번호
1          차량등록번호
2        본거지법정동코드
3           소유자구분
4         차종_등록정보
5              용도
6            차대번호
7          최초등록일자
8            차량연식
9          차량제작일자
10          검사유효일
11         제원관리번호
12       배출가스인증번호
13         배출가스등급
14        저공해조치종류
15    저공해조치구조변경YN
16         차량말소YN
17           취득일자
18           취득금액
19             차명
20        차종_제원정보
21           차종분류
22           차종유형
23          자동차형식
24           제작사명
25             연료
26           엔진형식
27            총중량
28           적재중량
29           엔진출력
30            배기량
31           승차인원
32           차량길이
33           차량너비
34           차량높이
35           원산지명
36           삭제YN
37         제원승인일자
38           구동형식
39          변속기종류
40           말소일자
41           말소코드
Name: 컬럼명, dtype: object

In [211]:
# 20.8s
ex = pd.read_table('data/220819_22.06월기준_말소.txt', header=42, names=ex_col, usecols=['차대번호', '차량말소YN'], encoding='cp949')
ex.columns

Index(['차대번호', '차량말소YN'], dtype='object')

In [212]:
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653054 entries, 0 to 3653053
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   차대번호    object
 1   차량말소YN  object
dtypes: object(2)
memory usage: 55.7+ MB


In [213]:
ex['차량말소YN'].unique()

array(['Y'], dtype=object)

## [로드] 컬럼명 데이터

In [214]:
col_df = pd.read_table('data/[제원정보]컬럼명.txt', sep=',')
col= col_df.columns

In [215]:
col

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

## [로드] 파일

In [216]:
# 22.3s
df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)
df.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15180\3437007485.py:2: DtypeWarning: Columns (14,17,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528586 entries, 0 to 1528585
Data columns (total 40 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1528586 non-null  object 
 1   차량등록번호       1528586 non-null  object 
 2   본거지법정동코드     1528586 non-null  int64  
 3   소유자구분        1528586 non-null  object 
 4   차종_등록정보      1528584 non-null  object 
 5   용도           1528586 non-null  object 
 6   차대번호         1528586 non-null  object 
 7   최초등록일자       1528586 non-null  int64  
 8   차량연식         1528586 non-null  int64  
 9   차량제작일자       1358925 non-null  float64
 10  검사유효일        1528574 non-null  float64
 11  제원관리번호       1528586 non-null  object 
 12  배출가스인증번호     1405585 non-null  object 
 13  배출가스등급       1528586 non-null  int64  
 14  저공해조치종류      195 non-null      object 
 15  저공해조치구조변경YN  1528586 non-null  object 
 16  차량말소YN       1528586 non-null  object 
 17  취득일자         1435514 non-null  object 
 18  취득

# 전처리

## 차량말소 정보 추가

In [217]:
df['차량말소YN'].unique()

array(['N'], dtype=object)

In [218]:
df2 = df.merge(ex, on='차대번호', how='left')
df2['차량말소YN_y'].unique()

array([nan, 'Y'], dtype=object)

In [219]:
df3 = df2[df2['차량말소YN_y'].isnull() == True].copy()
df3.shape

(1528579, 41)

In [220]:
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN_x', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN_y'],
      dtype='object')

In [221]:
df3 = df3.drop('차량말소YN_x', axis=1)
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN_y'],
      dtype='object')

In [222]:
df3.columns = ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN']

In [223]:
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN'],
      dtype='object')

## 차대번호 중복

In [224]:
len(df3['차대번호'].unique())

1528557

In [225]:
df3.shape

(1528579, 40)

In [226]:
df3.loc[df3.duplicated('차대번호', keep=False) == True].sort_values('차대번호')

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류,차량말소YN
747837,1GBEL19W3XB192382-02,39고6855,1141011800,주민,승용,자가용,1GBEL19W3XB192382,20091013,1999,19991231.0,...,6,4805.0,1980.0,1915.0,독일,N,20090805,NaN,자동,NaN
1424917,1GBEL19W3XB192382-01,36거2927,1111017500,주민,승용,자가용,1GBEL19W3XB192382,20090917,1999,20081124.0,...,6,4805.0,1980.0,1915.0,독일,N,20090805,NaN,자동,NaN
173765,2799대구2고2891,대구2고2891,2717010200,주민,승용,자가용,KLAEM19W1NB116889,19920127,1992,NaN,...,5,4800.0,1720.0,1417.0,대한민국,N,19910531,NaN,수동,NaN
1522531,1199서울2수2138,서울2수2138,2818510600,주민,승용,자가용,KLAEM19W1NB116889,19920127,1992,NaN,...,5,4800.0,1720.0,1417.0,대한민국,N,19910531.0,NaN,수동,NaN
639215,KLASM69F1JB054053-02,서울4모4528,1153010800,법인,승용,자가용,KLASM69F1JB054053,19880305,1988,NaN,...,5,4934.0,1720.0,1420.0,대한민국,N,19871205.0,NaN,수동,NaN
1022721,KLASM69F1JB054053-01,서울4수5512,1174010500,법인,승용,자가용,KLASM69F1JB054053,19880304,1988,NaN,...,5,4934.0,1720.0,1420.0,대한민국,N,19871205.0,NaN,수동,NaN
1486742,1199서울4드4983,서울4드4983,1138010300,주민,승용,자가용,KLATF19T1LB267581,19900124,1990,NaN,...,5,4390.0,1660.0,1360.0,NaN,N,19880805.0,NaN,수동,NaN
29276,1199서울4드4981,서울4드4981,1156010800,주민,승용,자가용,KLATF19T1LB267581,19900124,1990,NaN,...,5,4390.0,1660.0,1360.0,NaN,N,19880805,NaN,수동,NaN
659488,KLATF19T1NB526286-01,경남1우3305,4812515800,주민,승용,자가용,KLATF19T1NB526286,19920422,1992,NaN,...,5,4510.0,1663.0,1362.0,NaN,N,19910814.0,NaN,수동,NaN
682561,KLATF19T1NB526286-02,서울4쿠8476,1138011400,주민,승용,자가용,KLATF19T1NB526286,19930112,1992,NaN,...,5,4510.0,1663.0,1362.0,대한민국,N,19920605,NaN,수동,NaN


In [227]:
df3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN'],
      dtype='object')

In [228]:
df3.loc[df3.duplicated('차대번호', keep=False) == True, ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '원산지명', '제원승인일자']].sort_values('차대번호')

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,차명,차종_제원정보,차종분류,차종유형,자동차형식,제작사명,연료,엔진형식,원산지명,제원승인일자
747837,1GBEL19W3XB192382-02,39고6855,1141011800,주민,승용,자가용,1GBEL19W3XB192382,20091013,1999,19991231.0,...,시보레 아스트로 벤,승용,기타,대형,EL19W,오토라인,휘발유,4G,독일,20090805
1424917,1GBEL19W3XB192382-01,36거2927,1111017500,주민,승용,자가용,1GBEL19W3XB192382,20090917,1999,20081124.0,...,시보레 아스트로 벤,승용,기타,대형,EL19W,오토라인,휘발유,4G,독일,20090805
173765,2799대구2고2891,대구2고2891,2717010200,주민,승용,자가용,KLAEM19W1NB116889,19920127,1992,NaN,...,로얄프린스2.0MT,승용,일반,중형,EM19W,대우자동차(주),휘발유,C20LE,대한민국,19910531
1522531,1199서울2수2138,서울2수2138,2818510600,주민,승용,자가용,KLAEM19W1NB116889,19920127,1992,NaN,...,로얄프린스2.0MT,승용,일반,중형,EM19W,대우자동차(주),휘발유,C20LE,대한민국,19910531.0
639215,KLASM69F1JB054053-02,서울4모4528,1153010800,법인,승용,자가용,KLASM69F1JB054053,19880305,1988,NaN,...,로얄살롱2.0MT,승용,일반,중형,SM69F,대우자동차(주),휘발유,H20MR,대한민국,19871205.0
1022721,KLASM69F1JB054053-01,서울4수5512,1174010500,법인,승용,자가용,KLASM69F1JB054053,19880304,1988,NaN,...,로얄살롱2.0MT,승용,일반,중형,SM69F,대우자동차(주),휘발유,H20MR,대한민국,19871205.0
1486742,1199서울4드4983,서울4드4983,1138010300,주민,승용,자가용,KLATF19T1LB267581,19900124,1990,NaN,...,르망,승용,일반,소형,TF19I,대우자동차(주),휘발유,G15SF,NaN,19880805.0
29276,1199서울4드4981,서울4드4981,1156010800,주민,승용,자가용,KLATF19T1LB267581,19900124,1990,NaN,...,르망,승용,일반,소형,TF19I,대우자동차(주),휘발유,G15SF,NaN,19880805
659488,KLATF19T1NB526286-01,경남1우3305,4812515800,주민,승용,자가용,KLATF19T1NB526286,19920422,1992,NaN,...,르망,승용,일반,소형,TF19I,대우자동차(주),휘발유,G15SF,NaN,19910814.0
682561,KLATF19T1NB526286-02,서울4쿠8476,1138011400,주민,승용,자가용,KLATF19T1NB526286,19930112,1992,NaN,...,르망,승용,일반,소형,TF19I,대우자동차(주),휘발유,G15SF,대한민국,19920605


In [229]:
# 차대번호 중복 예시
df3.loc[df3.duplicated('차대번호', keep=False) == True, ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '원산지명', '제원승인일자']].sort_values('차대번호').to_excel('analysis/[G4][차대번호]중복_예시.xlsx')

In [230]:
df3.sort_values(['최초등록일자', '검사유효일', '차대번호'], ascending=False).head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류,차량말소YN
1528583,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,3,7710.0,2420.0,2960.0,대한민국,N,20080529.0,NaN,자동,NaN
1528561,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,2,6675.0,2255.0,2950.0,대한민국,N,20080518.0,NaN,수동,NaN
1528540,KLACA26RD8B226753-01,153구6446,2826011000,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,7,4635.0,1850.0,1720.0,대한민국,N,20070703.0,NaN,자동,NaN
1528538,KMCGK17HP8C086937-01,88조4842,4476032000,법인,화물,자가용,KMCGK17HP8C086937,20210830,2008,20080315.0,...,3,6180.0,2030.0,2920.0,대한민국,N,20070913.0,NaN,수동,NaN
1528539,KMCYSS7JP7U084697-01,99도4857,4423038028,주민,특수,자가용,KMCYSS7JP7U084697,20210826,2007,20070227.0,...,3,5435.0,1820.0,2060.0,대한민국,N,20060714.0,NaN,수동,NaN


In [231]:
df3.shape

(1528579, 40)

In [232]:
df3_1 = df3.sort_values(['최초등록일자', '검사유효일', '차대번호'], ascending=False).reset_index(drop=True)
df3_2 = df3_1.drop_duplicates('차대번호', keep='first').reset_index(drop=True)

In [233]:
df3_2['차대번호'].value_counts()

KMCLK18KP8C035195    1
KMHJN81VP8U766322    1
KNAJE55537K463438    1
KNAJE55537K468980    1
KNAJE55537K470819    1
                    ..
KMHNV81WBAU108544    1
KMHNV81WDAU113985    1
KMHSH81UBAU530489    1
KMHSH81UBAU541710    1
KMHJF31JPPU555457    1
Name: 차대번호, Length: 1528557, dtype: int64

In [234]:
df3_2[df3_2['차대번호'] == 'KNADA2322JS052345']['차명']

1523727    프라이드
Name: 차명, dtype: object

In [235]:
df4 = df3_2.copy()

## 연료 정보 수정
- 엔진형식, 차명으로 확인

In [236]:
f = df4.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1167605
5,휘발유,334494
1,LPG(액화석유가스),26418
3,기타연료,38
0,CNG(압축천연가스),1
4,알코올,1


In [237]:
g = df4.groupby('연료', as_index=False)['차대번호'].groups
g['기타연료']

Int64Index([   3059, 1486836, 1498416, 1499706, 1501418, 1502065, 1502241,
            1502591, 1503075, 1504853, 1506843, 1506983, 1507069, 1507214,
            1507417, 1507583, 1507880, 1508213, 1508592, 1508820, 1509438,
            1510445, 1511196, 1511799, 1513346, 1514189, 1516533, 1518320,
            1519203, 1519358, 1519683, 1520041, 1521843, 1522798, 1522963,
            1523260, 1524481, 1528151],
           dtype='int64')

In [238]:
# B5 : 휘발유
# G16SF : 휘발유
# H20MR : 휘발유
# MSV3 : 휘발유
# 01821300581 : 
# G15CF : 휘발유
# G4AJ : 휘발유
df4.loc[df4['연료'] == '기타연료', '엔진형식'].unique()

array(['-', 'MSV3', 'B5', 'H20MR', 'G16SF', 'G4AJ', nan, 'G15CF',
       '01821300581'], dtype=object)

In [239]:
df4.loc[df4['엔진형식'] == 'B5', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'G16SF', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'H20MR', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'MSV3', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'G15CF', '연료'] = '휘발유'
df4.loc[df4['엔진형식'] == 'G4AJ', '연료'] = '휘발유'
df4[df4['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
3059,에어스트림탐지견수송트레일러,-
1516533,스텔라오토매틱,NaN
1518320,벤즈 230E,NaN
1524481,벤즈230E,01821300581
1528151,도요다크라운로얄싸롱,NaN


In [240]:
df4.loc[df4['차명'] == '도요다크라운로얄싸롱', '연료'] = '휘발유'
df4.loc[df4['차명'] == '스텔라오토매틱', '연료'] = '휘발유'
df4.loc[df4['차명'] == '벤즈 230E', '연료'] = '휘발유'
df4.loc[df4['차명'] == '벤즈230E', '연료'] = '휘발유'
df4[df4['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
3059,에어스트림탐지견수송트레일러,-


In [241]:
f = df4.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1167605
5,휘발유,334531
1,LPG(액화석유가스),26418
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


In [242]:
f.columns = ['연료', '4등급']
f

,연료,4등급
2,경유,1167605
5,휘발유,334531
1,LPG(액화석유가스),26418
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


## [excel] 연료별 통계

In [38]:
f.to_excel('analysis/[G4]연료별_차대번호_통계.xlsx', index=False, encoding='utf-8')

## 지역별 수량 통계

## [로드] 법정동코드

In [243]:
code = pd.read_excel('data/법정동코드_2022.10.14.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


In [252]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y
2,1111010100,서울,종로구,청운동,NaN,NaN,존재,수도권,Y
3,1111010200,서울,종로구,신교동,NaN,NaN,존재,수도권,Y
4,1111010300,서울,종로구,궁정동,NaN,NaN,존재,수도권,Y


In [41]:
code['code1'] = code['법정동코드'].astype('str').str[:5]

In [42]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y,11000
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y,11110
2,1111010100,서울,종로구,청운동,NaN,NaN,존재,수도권,Y,11110
3,1111010200,서울,종로구,신교동,NaN,NaN,존재,수도권,Y,11110
4,1111010300,서울,종로구,궁정동,NaN,NaN,존재,수도권,Y,11110


In [43]:
code.shape

(46340, 10)

In [44]:
code2 = code.drop_duplicates(['시도', 'code1'], keep='first').reset_index(drop=True)
code2.shape

(461, 10)

In [45]:
code2.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
0,1100000000,서울,NaN,NaN,NaN,NaN,존재,수도권,Y,11000
1,1111000000,서울,종로구,NaN,NaN,NaN,존재,수도권,Y,11110
2,1114000000,서울,중구,NaN,NaN,NaN,존재,수도권,Y,11140
3,1117000000,서울,용산구,NaN,NaN,NaN,존재,수도권,Y,11170
4,1120000000,서울,성동구,NaN,NaN,NaN,존재,수도권,Y,11200


In [46]:
code2[code2['시도'] == '세종']

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN,code1
122,3611000000,세종,NaN,NaN,NaN,NaN,존재,중부권,Y,36110


In [47]:
code2['code1'].dtype

dtype('O')

In [48]:
code2 = code2[['시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN', 'code1']]

In [49]:
code2.head()

,시도,시구군,폐지여부,대기관리권역,대기관리권역YN,code1
0,서울,NaN,존재,수도권,Y,11000
1,서울,종로구,존재,수도권,Y,11110
2,서울,중구,존재,수도권,Y,11140
3,서울,용산구,존재,수도권,Y,11170
4,서울,성동구,존재,수도권,Y,11200


### 법정동 코드 추가

In [50]:
df4['본거지법정동코드'].head()

0    4824034000
1    4482533000
2    2826011000
3    4476032000
4    4423038028
Name: 본거지법정동코드, dtype: int64

In [51]:
df4['본거지법정동코드'].isnull().sum()

0

In [52]:
df4['code1'] = df4['본거지법정동코드'].astype('str').str[:5]
df4['code1'].head()

0    48240
1    44825
2    28260
3    44760
4    44230
Name: code1, dtype: object

In [53]:
df5 = df4.merge(code2, on='code1', how='left')
df5[['code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN']].head()

,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN
0,48240,경남,사천시,존재,동남권,N
1,44825,충남,태안군,존재,중부권,Y
2,28260,인천,서구,존재,수도권,Y
3,44760,충남,부여군,존재,중부권,Y
4,44230,충남,논산시,존재,중부권,Y


In [54]:
df5.shape

(1528557, 46)

In [55]:
df5['시도'].isnull().sum()

0

In [56]:
df5['시도'].unique()

array(['경남', '충남', '인천', '전북', '울산', '경북', '대전', '부산', '광주', '강원', '경기',
       '전남', '충북', '대구', '서울', '제주', '세종'], dtype=object)

In [57]:
df5['시구군'].isnull().sum()

8193

In [58]:
df5['시구군'].unique()

array(['사천시', '태안군', '서구', '부여군', '논산시', '김제시', '완주군', '울주군', '구미시',
       '고성군', '거제시', '울진군', '대덕구', '중구', '원주시', '성남시 중원구', '목포시', '완도군',
       '남양주시', '청주시 흥덕구', '춘천시', '단양군', '동구', '김천시', '전주시 덕진구', '함안군',
       '영주시', '안성시', '남구', '칠곡군', '사하구', '김포시', '서산시', '달서구', '군산시',
       '종로구', '성남시 수정구', '청주시 상당구', '인제군', '장성군', '광산구', '청도군', '경산시',
       '여수시', '천안시 서북구', '고양시 일산동구', '아산시', '동대문구', '창원시 의창구', '부천시',
       '홍성군', '구리시', '포천시', '철원군', '음성군', '창원시 진해구', '부평구', '양산시', '북구',
       '보은군', '여주시', '용인시 기흥구', '평택시', '광양시', '서천군', '서초구', '당진시', '은평구',
       '천안시 동남구', '고흥군', '달성군', '충주시', '양주시', '수원시 장안구', '파주시', '예산군',
       '강릉시', '강화군', '성동구', '노원구', '안산시 상록구', '성북구', '영등포구', '구로구', '고령군',
       '영동군', '제천시', '강남구', '강동구', '미추홀구', '광진구', '시흥시', '영암군', '연제구',
       '창녕군', '해운대구', '광주시', '영천시', '진주시', '제주시', '공주시', '마포구', '사상구',
       '용인시 처인구', '서대문구', '경주시', '화순군', '횡성군', '포항시 남구', '유성구', '나주시',
       '금정구', '청주시 청원구', '연수구', '수성구', '산청군', '광명시', '용산구', '밀양시', '양양군

In [59]:
df5.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [60]:
l = df5.groupby(['시도', '대기관리권역YN', '연료'])['차대번호'].count()
l

시도  대기관리권역YN  연료         
강원  N         LPG(액화석유가스)      806
              경유             47557
              휘발유             8645
경기  N         LPG(액화석유가스)      119
              경유              8029
                             ...  
충북  N         경유              9380
              휘발유             1603
    Y         LPG(액화석유가스)     1071
              경유             39955
              휘발유             9639
Name: 차대번호, Length: 78, dtype: int64

In [61]:
l.unstack()

연료           CNG(압축천연가스)  LPG(액화석유가스)        경유  기타연료  알코올      휘발유
시도 대기관리권역YN                                                        
강원 N                 NaN        806.0   47557.0   NaN  NaN   8645.0
경기 N                 NaN        119.0    8029.0   NaN  NaN   2018.0
   Y                 NaN       6852.0  263413.0   NaN  NaN  87318.0
경남 N                 NaN        319.0   35671.0   1.0  NaN   5813.0
   Y                 NaN        725.0   62151.0   NaN  NaN  13147.0
경북 N                 NaN        457.0   46305.0   NaN  NaN   6766.0
   Y                 NaN       1014.0   50059.0   NaN  NaN  13076.0
광주 Y                 NaN        721.0   30708.0   NaN  NaN   8419.0
대구 Y                 NaN       1675.0   51173.0   NaN  NaN  16943.0
대전 Y                 NaN       1248.0   30365.0   NaN  NaN  10577.0
부산 Y                 NaN       1517.0   58875.0   NaN  NaN  17442.0
서울 Y                 NaN       4439.0  112475.0   NaN  NaN  63609.0
세종 Y                 NaN         66.0    6790.0   NaN  NaN   1337.0
울산 Y                 NaN        273.0   26599.0   NaN  NaN   5964.0
인천 N                 NaN         14.0    1436.0   NaN  NaN    218.0
   Y                 NaN       1305.0   60230.0   NaN  1.0  18656.0
전남 N                 1.0        749.0   37746.0   NaN  NaN   5880.0
   Y                 NaN        532.0   34988.0   NaN  NaN   6761.0
전북 N                 NaN        399.0   29035.0   NaN  NaN   4261.0
   Y                 NaN        590.0   32030.0   NaN  NaN   7142.0
제주 N                 NaN        379.0   22055.0   NaN  NaN   4402.0
충남 N                 NaN         33.0    2372.0   NaN  NaN    459.0
   Y                 NaN        957.0   68208.0   NaN  NaN  14436.0
충북 N                 NaN        158.0    9380.0   NaN  NaN   1603.0
   Y                 NaN       1071.0   39955.0   NaN  NaN   9639.0

### [excel] 지역별 수량 통계

In [62]:
l.unstack().to_excel('analysis/[G4]지역별_차대번호_통계.xlsx', encoding='utf-8')

# [출력] 4등급 제원정보(csv)
- 차량말소 적용, 차대번호 중복제거, 연료정보 수정, 지역명 추가

In [63]:
# 4등급 제원정보 수정(차량말소 적용, 차대번호 중복제거, 연료정보 수정, 지역명 추가)
# 30.2s
df5.to_csv('data/220819_22.06월기준_4등급(수정).csv', encoding='cp949')

## 지역별 연료별 통계

In [64]:
df5.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [66]:
statics_by_local_fuel = df5.groupby(['시도', '연료'])['차대번호'].count()
statics_by_local_fuel

시도  연료         
강원  LPG(액화석유가스)       806
    경유              47557
    휘발유              8645
경기  LPG(액화석유가스)      6971
    경유             271442
    휘발유             89336
경남  LPG(액화석유가스)      1044
    경유              97822
    기타연료                1
    휘발유             18960
경북  LPG(액화석유가스)      1471
    경유              96364
    휘발유             19842
광주  LPG(액화석유가스)       721
    경유              30708
    휘발유              8419
대구  LPG(액화석유가스)      1675
    경유              51173
    휘발유             16943
대전  LPG(액화석유가스)      1248
    경유              30365
    휘발유             10577
부산  LPG(액화석유가스)      1517
    경유              58875
    휘발유             17442
서울  LPG(액화석유가스)      4439
    경유             112475
    휘발유             63609
세종  LPG(액화석유가스)        66
    경유               6790
    휘발유              1337
울산  LPG(액화석유가스)       273
    경유              26599
    휘발유              5964
인천  LPG(액화석유가스)      1319
    경유              61666
    알코올                 1
    휘발유             18

In [67]:
statics_by_local_fuel2 = statics_by_local_fuel.unstack(['연료'])
statics_by_local_fuel2

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
시도,,,,,,
강원,NaN,806.0,47557.0,NaN,NaN,8645.0
경기,NaN,6971.0,271442.0,NaN,NaN,89336.0
경남,NaN,1044.0,97822.0,1.0,NaN,18960.0
경북,NaN,1471.0,96364.0,NaN,NaN,19842.0
광주,NaN,721.0,30708.0,NaN,NaN,8419.0
대구,NaN,1675.0,51173.0,NaN,NaN,16943.0
대전,NaN,1248.0,30365.0,NaN,NaN,10577.0
부산,NaN,1517.0,58875.0,NaN,NaN,17442.0
서울,NaN,4439.0,112475.0,NaN,NaN,63609.0


In [68]:
statics_by_local_fuel2.to_excel('analysis/[G4][법정동코드]지역_연료별_통계.xlsx', encoding='utf-8')

# 분석

## 배출가스

### 배출가스인증번호 내역 확인

In [69]:
df5['배출가스인증번호'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1528557 entries, 0 to 1528556
Series name: 배출가스인증번호
Non-Null Count    Dtype 
--------------    ----- 
1405566 non-null  object
dtypes: object(1)
memory usage: 23.3+ MB


In [70]:
df5['배출가스인증번호'].isnull().sum()

122991

In [71]:
df5['배출가스인증번호'].value_counts()

7MY-HD-14-68     268972
8MY-KM-13-19      93969
6MY-KM-14-61      76370
9MY-KM-14-41      67493
7MY-HD-14-46      60364
                  ...  
5MY-DC-23-06          1
8MY-DH-0M-24          1
 7MY-DC-24-72         1
8MY-WI-0M-24          1
7MY-MM-0M-24          1
Name: 배출가스인증번호, Length: 304, dtype: int64

In [72]:
temp = df5['배출가스인증번호'].value_counts().reset_index()
temp

,index,배출가스인증번호
0,7MY-HD-14-68,268972
1,8MY-KM-13-19,93969
2,6MY-KM-14-61,76370
3,9MY-KM-14-41,67493
4,7MY-HD-14-46,60364
...,...,...
299,5MY-DC-23-06,1
300,8MY-DH-0M-24,1
301,7MY-DC-24-72,1
302,8MY-WI-0M-24,1


In [73]:
# 제원정보 배출가스인증번호 현황
temp.to_csv('analysis/[G4][배출가스인증번호]유니크값_통계.csv', encoding='cp949')

In [74]:
df5.shape

(1528557, 46)

#### 12\. [e]배출가스인증번호

In [75]:
# - 정보
#     - 총 : 1528557
#     - null : 123003
#     - 정상
#         - 2005.01.04 이후 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}-[숫자]{2}, 12자리) : 1152678
#         - 2005.01.04 이전 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}, 9자리) : 230866
#     - 오류(null 값 제외) : 22024
#       - 예시
#           - 163      TY87-배K-02
#           - 173    7MY-KJ-0M-24
#           - 216      TY87-배K-03
#           - 236      TY87-배K-10
#           - 293      TY87-배K-03

In [76]:
# 2005년 이후 표기 방식 중 길이가 12자리가 아닌 값 : 3018
#     - [max] 차량연식 : 2011
# 2005년 이전 표기 방식 중 길이가 9자리가 아닌 값 : 2

##### 빈칸 처리

In [77]:
gas_df = df5.copy()
gas_df.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,제원승인일자,구동형식,변속기종류,차량말소YN,code1,시도,시구군,폐지여부,대기관리권역,대기관리권역YN
0,KMCLK19KP8C035195-01,96부7399,4824034000,주민,화물,자가용,KMCLK18KP8C035195,20220614,2008,20080924.0,...,20080529.0,NaN,자동,NaN,48240,경남,사천시,존재,동남권,N
1,KMFLA18KP8C031274-01,92도0997,4482533000,법인,화물,자가용,KMFLA18KP8C031274,20220419,2008,20080605.0,...,20080518.0,NaN,수동,NaN,44825,충남,태안군,존재,중부권,Y
2,KLACA26RD8B226753-01,153구6446,2826011000,주민,승용,자가용,KLACA26RD8B226753,20210909,2008,20080221.0,...,20070703.0,NaN,자동,NaN,28260,인천,서구,존재,수도권,Y
3,KMCGK17HP8C086937-01,88조4842,4476032000,법인,화물,자가용,KMCGK17HP8C086937,20210830,2008,20080315.0,...,20070913.0,NaN,수동,NaN,44760,충남,부여군,존재,중부권,Y
4,KMCYSS7JP7U084697-01,99도4857,4423038028,주민,특수,자가용,KMCYSS7JP7U084697,20210826,2007,20070227.0,...,20060714.0,NaN,수동,NaN,44230,충남,논산시,존재,중부권,Y


In [78]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호']

8808        
12687       
1194061     
1236423     
1336420     
1399623     
1477301     
1492213     
1507852     
1514578     
1518320     
1528151     
Name: 배출가스인증번호, dtype: object

In [79]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].shape

(12,)

In [80]:
blank_index = gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].index
blank_index

Int64Index([   8808,   12687, 1194061, 1236423, 1336420, 1399623, 1477301,
            1492213, 1507852, 1514578, 1518320, 1528151],
           dtype='int64')

In [81]:
# 빈칸을 포함한 예시
gas_df.iloc[103131, 12]

'7MY-HD-14-68'

In [82]:
gas_df.loc[blank_index, '배출가스인증번호'] = np.nan
gas_df.iloc[103131, 12]

'7MY-HD-14-68'

In [83]:
gas_df['배출가스인증번호'].isnull().sum()

123003

In [84]:
df5.iloc[:, 12].value_counts()

7MY-HD-14-68     268972
8MY-KM-13-19      93969
6MY-KM-14-61      76370
9MY-KM-14-41      67493
7MY-HD-14-46      60364
                  ...  
5MY-DC-23-06          1
8MY-DH-0M-24          1
 7MY-DC-24-72         1
8MY-WI-0M-24          1
7MY-MM-0M-24          1
Name: 배출가스인증번호, Length: 304, dtype: int64

In [85]:
df5.iloc[:, 12].isnull().sum()

122991

In [86]:
df5[df5['배출가스인증번호'].isnull() == True]['배출가스인증번호'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: 배출가스인증번호, dtype: object

In [87]:
# 2005.01.04 이후 표기 방식 : 1152678
count = 0
p121 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}[-][0-9]{2}')
idx121 = []
for i, one in tqdm(enumerate(gas_df.iloc[:, 12].to_numpy())):
    if p121.search(str(one)):
        idx121.append(i)
        count += 1
count

1528557it [00:01, 859669.37it/s] 


1152678

In [88]:
# 2005.01.04 이후 표기 방식이 아닌 값 : 375879
count = 0
idx122 = []
for i, one in tqdm(enumerate(gas_df.iloc[:, 12].to_numpy())):
    if not p121.search(str(one)):
        idx122.append(i)
        count += 1
count

1528557it [00:01, 1020724.32it/s]


375879

In [89]:
gas_df.iloc[:, 12].reindex(idx122)[:5]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: 배출가스인증번호, dtype: object

In [90]:
# 2005.01.04 이전 배출가스 표기 방식 : 230852
count = 0
p123 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}')
idx123 = []
for i, one in tqdm(enumerate(gas_df.iloc[idx122, 12].to_numpy())):
    if p123.search(str(one)):
        idx123.append(idx122[i])
        count += 1
count

375879it [00:00, 842992.79it/s]


230852

In [91]:
gas_df.iloc[idx123, [7, 8, 9, 17, 12]][:5]

,최초등록일자,차량연식,차량제작일자,취득금액,배출가스인증번호
936,20171018,1997,19971231.0,116130.0,RMY-HD-20
970,20170918,1998,19981231.0,200000.0,VMY-HD-30
1061,20170721,1999,19991231.0,0.0,WMY-HD-32
1066,20170720,2002,20021231.0,700000.0,WMY-HD-32
2307,20150825,1999,19991231.0,1195300.0,VMY-HD-30


In [92]:
# null 값 : 123003
idx124 = gas_df[gas_df['배출가스인증번호'].isnull() == True]['배출가스인증번호'].index
len(idx124)

123003

In [93]:
gas_df.iloc[:, 12].reindex(idx124)[:5]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: 배출가스인증번호, dtype: object

In [94]:
# 2005.01.04 이후 표기 방식이 아닌 값 중 null 값 제외한 오류 값 : 22024
idx125 = sorted(set(idx122) - set(idx124) - set(idx123))
len(idx125)

22024

In [95]:
gas_df.iloc[:, 12].reindex(idx125)[:5]

30     6MY-SS-0M-24
104    FMY-VTK-14-1
131    FMY-VTK-14-1
133    6MY-SS-0M-24
134    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [96]:
# 2005년 이상 배출가스인증번호 데이터
gas_temp = gas_df.iloc[idx123, [7, 8, 9, 17, 12]]
gas_temp[gas_temp['차량연식'] >= 2005].head()

,최초등록일자,차량연식,차량제작일자,취득금액,배출가스인증번호
8096,20120409,2011,20111219.0,46363636.0,8MY-PA-14-3
14866,20120111,2011,20111219.0,2040000.0,8MY-PA-14-3
15195,20120109,2011,20110517.0,16250000.0,8MY-PA-14-3
15745,20120106,2011,20110216.0,9545455.0,8MY-PA-14-3
19226,20120102,2011,20110516.0,46363636.0,8MY-PA-14-3


In [97]:
# 2005년 이상년도에서 맞지않는 배출가스인증번호 샘플 : 3018
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12)]

,최초등록일자,차량연식,차량제작일자,취득금액,배출가스인증번호
8096,20120409,2011,20111219.0,46363636.0,8MY-PA-14-3
14866,20120111,2011,20111219.0,2040000.0,8MY-PA-14-3
15195,20120109,2011,20110517.0,16250000.0,8MY-PA-14-3
15745,20120106,2011,20110216.0,9545455.0,8MY-PA-14-3
19226,20120102,2011,20110516.0,46363636.0,8MY-PA-14-3
...,...,...,...,...,...
1168976,20051104,2005,20050905.0,34545000.0,4MY-PA-06
1168984,20051103,2005,20051024.0,17660000.0,4MY-PA-06
1169185,20051014,2005,20050903.0,2983000.0,4MY-PA-06
1169636,20050823,2005,20050421.0,3100000.0,4MY-AD-20


In [98]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 )].shape

(3018, 5)

In [99]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 ), '차량연식'].max()

2011

In [100]:
# 2005년 미만 년도에서 맞지않는 배출가스인증번호 샘플 : 2
gas_temp.loc[(gas_temp['차량연식'] < 2005) & (gas_temp['배출가스인증번호'].str.len() != 9 )]

,최초등록일자,차량연식,차량제작일자,취득금액,배출가스인증번호
534831,20091013,1999,19991231.0,5000000.0,9MY-PP-14-W38


In [101]:
num_by_gas_num = gas_df.groupby('배출가스인증번호')['차대번호'].count().reset_index()
num_by_gas_num

,배출가스인증번호,차대번호
0,7MY-DC-24-72,1
1,7MY-HD-24-30,8
2,8MY-DC-24-12,1
3,3MY-FK-08,1067
4,4MY-AD-20,38
...,...,...
298,XMY-HD-47,924
299,XMY-HD-48,364
300,XMY-HD-55,2532
301,XMY-KM-47,3217


In [102]:
num_by_gas_num.to_excel('analysis/[G4][배출가스인증번호]유니크값_통계.xlsx', encoding='utf-8')

In [103]:
gas_df['배출가스인증번호'].isnull().sum()

123003

##### 오류 배출가스인증번호 유형 통계

In [104]:
# - 총 오류 샘플 수 : 22024
#     - 1번째 부분 3자리가 아닌 경우 : 17840
#         - 빈 값(' ')인 경우 : 0
#         - 바('-') 기호가 없는 경우 : 6
#         - 1번째 부분 1자리인 경우(n-XXX-XX-XX) : 0
#         - 1번째 부분 2자리인 경우(nn-XXX-XX-XX) : 0
#         - 1번째 부분 4자리인 경우(nnnn-XXX-XX-XX) : 17834(17840)
#
#     - 2번째 부분 2자리가 아닌 경우 : 2827
#         - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
#         - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
#
#     - 2번째 부분 2자리이면서 숫자만 있는 경우 : 0
#
#     - 3번째 부분 2자리가 아닌 경우 : 1
#         - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 1
#
#     - 3번째 부분 2자리이면서 문자가 있는 경우 : 1356(19191)
#
#     - 4번째 부분 2자리가 아닌 경우 : 0(1999)
#         - 중복 오류 : 1999
#     - 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19191)
#         - 중복 오류 : 19191

In [105]:
gas_df.iloc[idx125[:5], 12]

30     6MY-SS-0M-24
104    FMY-VTK-14-1
131    FMY-VTK-14-1
133    6MY-SS-0M-24
134    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [106]:
gas_temp2 = gas_df.iloc[idx125, 12]
gas_temp2.head()

30     6MY-SS-0M-24
104    FMY-VTK-14-1
131    FMY-VTK-14-1
133    6MY-SS-0M-24
134    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

###### 1번째 자리 오류 유형

In [107]:
# 1번째 부분 3자리가 아닌 경우 : 17840
#     - 빈 값(' ')인 경우 : 0
#     - 바('-') 기호가 없는 경우 : 6
#     - 1자리인 경우 : 0
#     - 2자리인 경우 : 0
#     - 4자리인 경우 : 17836

In [108]:
# 1번째 부분 3자리가 아닌 경우 : 17840
idx126 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) != 3:
            idx126.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 415502.52it/s]


17840

In [109]:
gas_df.iloc[idx126, 12]

877898           C135
891638           C135
898807           C135
915591           C135
960831           C135
              ...    
1528547    TY87-배K-03
1528548    TY87-배K-03
1528549    TY87-배K-03
1528551    TY87-배K-03
1528552    TY87-배K-03
Name: 배출가스인증번호, Length: 17840, dtype: object

In [110]:
gas_df.iloc[idx126[0], 12]

'C135'

In [111]:
# 빈 값(' ')인 경우 : 0
count = 0
idx126_1 = []
for i, one in enumerate(gas_df.iloc[idx126, 12]):
    if one == ' ':
        idx126_1.append(gas_df.iloc[idx126, 12].index[i])
        count += 1
count

0

In [112]:
gas_df.iloc[idx126_1, 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [113]:
# 바('-') 기호가 없는 경우 : 6
count = 0
idx126_2 = []
for i, one in enumerate(gas_df.iloc[idx126, 12]):
    if '-' not in one:
        idx126_2.append(gas_df.iloc[idx126, 12].index[i])
        count += 1
count

6

In [114]:
gas_df.iloc[list(set(idx126_2) - set(idx126_1)), 12]

973286    C135
915591    C135
891638    C135
877898    C135
898807    C135
960831    C135
Name: 배출가스인증번호, dtype: object

In [115]:
len(list(set(idx126_2) - set(idx126_1)))

6

In [116]:
# 1번째 부분 1자리인 경우(n-XXX-XX-XX) : 0
idx126_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 1:
            idx126_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 759316.03it/s]


0

In [117]:
gas_df.iloc[idx126_3[:5], 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [118]:
set(idx126_1) == set(idx126_3)

True

In [119]:
# 1번째 부분 2자리인 경우(nn-XXX-XX-XX) : 0
idx126_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 2:
            idx126_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 917569.10it/s]


0

In [120]:
# 1번째 부분 4자리인 경우(nnnn-XXX-XX-XX) : 17834(17840)
idx126_5 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 4:
            idx126_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 454339.27it/s]


17840

In [121]:
# 4자리인 경우(앞에서 중복된 샘플 제거) : 17834
len(set(idx126_5) - set(idx126_1) - set(idx126_2))

17834

###### 2번째 자리 오류 유형

In [122]:
# 2번째 부분 2자리가 아닌 경우 : 2827
#   - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
#   - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 0

In [123]:
# 2번째 부분 2자리가 아닌 경우 : 2827
idx127 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) != 2:
            idx127.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 667283.21it/s]


2827

In [124]:
len(set(idx127) - set(idx126))

2827

In [125]:
gas_df.iloc[idx127[:5], 12]

104    FMY-VTK-14-1
131    FMY-VTK-14-1
134    FMY-VTK-14-1
135    FMY-VTK-14-1
137    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [126]:
# 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
idx127_1 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 1:
            idx127_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 956989.77it/s]


0

In [127]:
gas_df.iloc[idx127_1[:5], 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [128]:
# 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
idx127_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 3:
            idx127_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 709902.49it/s]


2827

In [129]:
gas_df.iloc[idx127_3[:5], 12]

104    FMY-VTK-14-1
131    FMY-VTK-14-1
134    FMY-VTK-14-1
135    FMY-VTK-14-1
137    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [130]:
# 2번째 부분 4자리인 경우(XXX-nnnn-XX-XX) : 0
idx127_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 4:
            idx127_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 760472.47it/s]


0

In [131]:
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 0
idx127_5 = []
p127_5 = re.compile('[0-9]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p127_5.search(str(one).split('-')[1]):
            idx127_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 564479.34it/s]


0

###### 3번째 자리 오류 유형

In [132]:
# 3번째 부분 2자리가 아닌 경우 : 1
#   - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 1
# 3번째 부분 2자리이면서 문자가 있는 경우 : 1356

In [133]:
# 3번째 부분 2자리가 아닌 경우 : 1
idx128 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[2]) != 2:
            idx128.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 917605.56it/s]


1

In [134]:
gas_df.iloc[idx128, 12]

1187545    NMY-JK-S-107
Name: 배출가스인증번호, dtype: object

In [135]:
len(set(idx128) - set(idx126) - set(idx127))

1

In [136]:
# 3번째 부분 2자리이면서 문자가 있는 경우 : 1356(19191)
idx128_1 = []
p128_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[2]) == 2) and p128_1.search(str(one).split('-')[1]):
            idx128_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 311956.64it/s]


22017

In [137]:
# 3번째 부분 2자리이면서 문자가 있는 경우(앞에서 중복된 샘플 제거) : 1356
len(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))

1356

In [138]:
gas_df.iloc[list(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))[:5], 12]

958464    7MY-HC-0M-24
933891    8MY-HS-0M-24
901127    7MY-CH-0M-24
958472    6MY-SS-0M-24
958477    8MY-HT-0M-24
Name: 배출가스인증번호, dtype: object

###### 4번째 자리 오류 유형

In [139]:
# 4번째 부분 2자리가 아닌 경우 : 0(1999)
#   - 중복 오류 : 1999
# 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19191)

In [140]:
# 4번째 부분 2자리가 아닌 경우 : 0(1999)
idx129 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[3]) != 2:
            idx129.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 733743.34it/s]


1999

In [141]:
gas_df.iloc[idx129[:5], 12]

104    FMY-VTK-14-1
131    FMY-VTK-14-1
134    FMY-VTK-14-1
135    FMY-VTK-14-1
137    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [142]:
# 중복 추출된 샘플 제거
len(set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

In [143]:
# 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19191)
idx129_1 = []
p129_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p129_1.search(str(one).split('-')[1]):
            idx129_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 323770.73it/s]


19191

In [144]:
len(set(idx129_1) - set(idx126) - set(idx127) - set(idx128_1))

1

###### 한글이 포함된 경우

In [145]:
idx1210 = []
p1210 = re.compile('[가-힣]')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if p1210.search(one):
            idx1210.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22024it [00:00, 512084.04it/s]


17834

In [146]:
len(set(idx1210) - set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

#### 배출가스인증번호 미부여 현황
- 필요한 컬럼 : 배출가스인증번호, 차량연식, 차량제작일자, 차명, 차종_제원정보, 차종분류, 차종유형, 자동차형식, 제작사명, 연료

In [147]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [148]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['배출가스인증번호', '차량연식', '차량제작일자', '차명', '차종_제원정보', '차종분류', '차종유형', '제작사명', '연료']].head()

,배출가스인증번호,차량연식,차량제작일자,차명,차종_제원정보,차종분류,차종유형,제작사명,연료
0,NaN,2008,20080924.0,메가트럭군물자수송,화물,특수용도형-특수용도형,대형,현대자동차(주),경유
1,NaN,2008,20080605.0,신정4㎥급진공노면총소차,화물,특수용도형-특수용도형,대형,신정개발특장차 주식회사,경유
2,NaN,2008,20080221.0,윈스톰2.0S디젤,승용,다목적,중형,한국지엠주식회사,경유
3,NaN,2008,20080315.0,마이티냉동차,화물,특수용도형-특수용도형,중형,현대자동차(주),경유
4,NaN,2007,20070227.0,리베로슈퍼렉커(LIBERO),특수,구난,중형,현대자동차(주),경유


In [149]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [150]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류']].shape

(123003, 40)

In [151]:
null_num_df = gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류']]

In [152]:
# [출력]4등급 차량 배출가스인증번호 미부여 샘플
null_num_df.to_csv('analysis/[G4][배출가스인증번호]미부여_샘플.csv', encoding='cp949')

In [153]:
null_num_df[['엔진형식', '차명', '차량연식','차대번호']].head()

,엔진형식,차명,차량연식,차대번호
0,D6GA,메가트럭군물자수송,2008,KMCLK18KP8C035195
1,D6GA,신정4㎥급진공노면총소차,2008,KMFLA18KP8C031274
2,Z20S,윈스톰2.0S디젤,2008,KLACA26RD8B226753
3,D4GA,마이티냉동차,2008,KMCGK17HP8C086937
4,D4CB,리베로슈퍼렉커(LIBERO),2007,KMCYSS7JP7U084697


In [154]:
null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False)

엔진형식          차명                   
B3            프라이드                     9885
F8C           티코(TICO)                 9435
A15MF         에스페로1.5MT                4941
C18LE         프린스1.8AT                 4591
              프린스1.8MT                 4277
                                       ... 
62M           PORSCHE 911 CARRERA 2       1
              PORSCHE 911카레라2             1
              포르쉐911카레라                   1
              포르쉐911카레라카브리올레              1
vg30-858076a  닛산세드릭                       1
Name: 차대번호, Length: 3103, dtype: int64

In [155]:
null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False).to_excel('analysis/[G4][배출가스인증번호]미부여샘플_통계.xlsx')

##### 배출가스인증번호 미부여 데이터 확인

In [156]:
# 배출가스인증번호 미부여 데이터 중
# - 동일 엔진형식 중 1천대 이상의 샘플만 배출가스인증번호 유무 확인

In [157]:
miss_gas_num_df = null_num_df.groupby(['엔진형식', '차명'])['차대번호'].count().sort_values(ascending=False).reset_index()
miss_gas_num_df

,엔진형식,차명,차대번호
0,B3,프라이드,9885
1,F8C,티코(TICO),9435
2,A15MF,에스페로1.5MT,4941
3,C18LE,프린스1.8AT,4591
4,C18LE,프린스1.8MT,4277
...,...,...,...
3098,62M,PORSCHE 911 CARRERA 2,1
3099,62M,PORSCHE 911카레라2,1
3100,62M,포르쉐911카레라,1
3101,62M,포르쉐911카레라카브리올레,1


In [158]:
temp = miss_gas_num_df[miss_gas_num_df['차대번호'] >= 1000]
temp

,엔진형식,차명,차대번호
0,B3,프라이드,9885
1,F8C,티코(TICO),9435
2,A15MF,에스페로1.5MT,4941
3,C18LE,프린스1.8AT,4591
4,C18LE,프린스1.8MT,4277
5,F8C,티코(TICO)오토,2710
6,G15SF,르망,2571
7,C20LE,수퍼살롱2.0AT,2299
8,A15DMS,누비라1.5,2122
9,G4AJ,프레스토,2074


In [159]:
# 제원정보에서 해당 엔진형식을 통해 배출가스인증번호 확인 여부 확인
count = 0
for one, two in zip(temp['엔진형식'], temp['차명']):
    if gas_df.loc[ (gas_df['엔진형식'] == one) & (gas_df['차명'] == two), ['엔진형식' ,'배출가스인증번호', '차명'] ].drop_duplicates(['배출가스인증번호', '차명'], keep='first').dropna(subset=['배출가스인증번호']).shape[0] != 0:
        print(one)
        print( gas_df.loc[ (gas_df['엔진형식'] == one) & (gas_df['차명'] == two), ['엔진형식' ,'배출가스인증번호', '차명'] ].drop_duplicates(['배출가스인증번호', '차명'], keep='first').dropna(subset=['배출가스인증번호']))
        print()
        count += 1 

B3
        엔진형식    배출가스인증번호    차명
1480586   B3  TY87-배K-03  프라이드

G15SF
          엔진형식    배출가스인증번호  차명
1349480  G15SF  TY87-배K-02  르망

G4CS
         엔진형식   배출가스인증번호      차명
1223970  G4CS  LMY-HD-08  그랜저2.4

B3
        엔진형식   배출가스인증번호   차명
1225014   B3  PMY-KM-20  아벨라

CD800
          엔진형식   배출가스인증번호   차명
1264214  CD800  SMY-KM-04  타우너

D5244T
          엔진형식     배출가스인증번호        차명
268522  D5244T  6MY-PA-14-4  볼보 S80D5



In [160]:
count

6

In [161]:
gas_df.loc[gas_df['엔진형식'] == 'B3', ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명'])

,엔진형식,배출가스인증번호,차명
211236,B3,PMY-KM-20,KIA AVELLA
1170976,B3,NaN,프라이드베타
1171409,B3,NaN,아벨라
1173517,B3,NaN,프라이드
1183908,B3,NaN,프라이드웨곤
1189503,B3,NaN,프라이드오토
1200996,B3,NaN,프라이드프렌드
1201892,B3,PMY-KM-20,델타
1205133,B3,NaN,프라이드밴
1212358,B3,NaN,프라이드장애자차


In [162]:
gas_df.loc[gas_df['엔진형식'] == 'B3', ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명']).dropna(subset=['배출가스인증번호'])

,엔진형식,배출가스인증번호,차명
211236,B3,PMY-KM-20,KIA AVELLA
1201892,B3,PMY-KM-20,델타
1225014,B3,PMY-KM-20,아벨라
1227811,B3,PMY-KM-20,아벨라운전교습차
1228534,B3,PMY-KM-20,아벨라델타운전교습차
1305234,B3,TY87-배K-03,프라이드베타오토
1373305,B3,RMY-KM-25,프라이드베타
1376211,B3,TY87-배K-03,프라이드베타
1480586,B3,TY87-배K-03,프라이드


In [163]:
# 제원정보에서 해당 엔진형식을 통해 배출가스인증번호 확인 여부 확인
t_temp = pd.DataFrame()
for one in temp['엔진형식']:
    temp = gas_df.loc[gas_df['엔진형식'] == one, ['엔진형식' ,'배출가스인증번호', '차명']].drop_duplicates(['배출가스인증번호', '차명']).dropna(subset=['배출가스인증번호'])
    if temp.shape[0] != 0:
        t_temp = pd.concat([t_temp, temp], ignore_index=True)

In [164]:
t_temp

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,PMY-KM-20,델타
2,B3,PMY-KM-20,아벨라
3,B3,PMY-KM-20,아벨라운전교습차
4,B3,PMY-KM-20,아벨라델타운전교습차
5,B3,TY87-배K-03,프라이드베타오토
6,B3,RMY-KM-25,프라이드베타
7,B3,TY87-배K-03,프라이드베타
8,B3,TY87-배K-03,프라이드
9,G15SF,TY87-배K-02,르망


In [165]:
t_temp.drop_duplicates(['엔진형식', '배출가스인증번호', '차명']).shape

(36, 3)

In [166]:
t_temp = t_temp.drop_duplicates(['엔진형식', '배출가스인증번호', '차명'])
t_temp

,엔진형식,배출가스인증번호,차명
0,B3,PMY-KM-20,KIA AVELLA
1,B3,PMY-KM-20,델타
2,B3,PMY-KM-20,아벨라
3,B3,PMY-KM-20,아벨라운전교습차
4,B3,PMY-KM-20,아벨라델타운전교습차
5,B3,TY87-배K-03,프라이드베타오토
6,B3,RMY-KM-25,프라이드베타
7,B3,TY87-배K-03,프라이드베타
8,B3,TY87-배K-03,프라이드
9,G15SF,TY87-배K-02,르망


In [167]:
# 미부여 데이터 처리 가능 샘플 추출
t_temp.to_excel('analysis/[G4][배출가스인증번호]미부여_처리용_샘플.xlsx', index=False)

### 배출가스인증번호별 통계

In [168]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [169]:
gas_df.shape

(1528557, 46)

In [170]:
len(gas_df['차대번호'].unique())

1528557

In [171]:
gas_df.loc[gas_df.duplicated('차대번호', keep=False) == True, ['차량관리번호', '차량등록번호', '차대번호', '배출가스인증번호']].sort_values('차대번호')

,차량관리번호,차량등록번호,차대번호,배출가스인증번호


In [172]:
gas_df.groupby('배출가스인증번호')['차대번호'].count()

배출가스인증번호
 7MY-DC-24-72       1
 7MY-HD-24-30       8
 8MY-DC-24-12       1
3MY-FK-08        1067
4MY-AD-20          38
                 ... 
XMY-HD-47         924
XMY-HD-48         364
XMY-HD-55        2532
XMY-KM-47        3217
YKM-KM-12          70
Name: 차대번호, Length: 303, dtype: int64

In [173]:
gas_df['배출가스인증번호'].isnull().sum()

123003

In [174]:
gas_df.groupby('배출가스인증번호')['차대번호'].count().to_excel('analysis/[G4][배출가스인증번호]번호별_통계.xlsx')

### 총중량으로 분류 현황

In [175]:
gas_df.shape

(1528557, 46)

In [176]:
gas_df['총중량'].unique()

array([10825., 11075.,  2215., ...,  1512.,  1197.,  1436.])

In [177]:
type(gas_df['총중량'][0])

numpy.float64

In [178]:
gas_df['총중량'].isnull().sum()

1

In [179]:
gas_df['총중량'] = gas_df['총중량'].fillna(-1)

In [180]:
gas_df['총중량'].isnull().sum()

0

In [181]:
weight_list = []
for i, one in tqdm(enumerate(gas_df['총중량'])):
    if one == -1:
        weight_list.append('미기입')
    elif one < 1500:
        weight_list.append('1.5ton 미만')
    elif one >= 1500 and one < 2000:
        weight_list.append('1.5ton 이상 2.0ton 미만')
    elif one >= 2000 and one < 2500:
        weight_list.append('2.0ton 이상 2.5ton 미만')
    elif one >= 2500 and one < 3000:
        weight_list.append('2.5ton 이상 3.0ton 미만')
    elif one >= 3000 and one < 3500:
        weight_list.append('3.0ton 이상 3.5ton 미만')
    elif one >= 3500 and one < 4000:
        weight_list.append('3.5ton 이상 4.0ton 미만')
    elif one >= 4000 and one < 4500:
        weight_list.append('4.0ton 이상 4.5ton 미만')
    elif one >= 4500 and one < 5000:
        weight_list.append('4.5ton 이상 5.0ton 미만')
    elif one >= 5000 and one < 5500:
        weight_list.append('5.0ton 이상 5.5ton 미만')
    elif one >= 5500 and one < 6000:
        weight_list.append('5.5ton 이상 6.0ton 미만')
    elif one >= 6000 and one < 8000:
        weight_list.append('6.0ton 이상 8.0ton 미만')
    elif one >= 8000 and one < 10000:
        weight_list.append('8.0ton 이상 10.0ton 미만')
    elif one >= 10000:
        weight_list.append('10.0ton 이상')

1528557it [00:01, 1057647.49it/s]


In [182]:
len(weight_list)

1528557

In [183]:
gas_df['총중량_분류'] = weight_list

In [184]:
total_weight_df = gas_df.groupby('총중량_분류')['차대번호'].count().reset_index()
total_weight_df

,총중량_분류,차대번호
0,1.5ton 미만,198424
1,1.5ton 이상 2.0ton 미만,330997
2,10.0ton 이상,30469
3,2.0ton 이상 2.5ton 미만,240259
4,2.5ton 이상 3.0ton 미만,436562
5,3.0ton 이상 3.5ton 미만,262435
6,3.5ton 이상 4.0ton 미만,135
7,4.0ton 이상 4.5ton 미만,192
8,4.5ton 이상 5.0ton 미만,23
9,5.0ton 이상 5.5ton 미만,891


In [185]:
total_weight_df.to_excel('analysis/[G4][총중량]분류_통계.xlsx', encoding='utf-8')

#### 총중량 code 화

In [186]:
weight_code_list = []
for i, one in tqdm(enumerate(gas_df['총중량'])):
    if one < 250:
        weight_code_list.append('0025')
    elif one >= 250 and one < 500:
        weight_code_list.append('0050')
    elif one >= 500 and one < 750:
        weight_code_list.append('0075')
    elif one >= 750 and one < 1000:
        weight_code_list.append('0100')
    elif one >= 1000 and one < 1250:
        weight_code_list.append('0125')
    elif one >= 1250 and one < 1500:
        weight_code_list.append('0150')
    elif one >= 1500 and one < 1750:
        weight_code_list.append('0175')
    elif one >= 1750 and one < 2000:
        weight_code_list.append('0200')
    elif one >= 2000 and one < 2250:
        weight_code_list.append('0225')
    elif one >= 2250 and one < 2500:
        weight_code_list.append('0250')
    elif one >= 2500 and one < 2750:
        weight_code_list.append('0275')
    elif one >= 2750 and one < 3000:
        weight_code_list.append('0300')
    elif one >= 3000 and one < 3250:
        weight_code_list.append('0325')
    elif one >= 3250 and one < 3500:
        weight_code_list.append('0350')
    elif one >= 3500 and one < 3750:
        weight_code_list.append('0375')
    elif one >= 3750 and one < 4000:
        weight_code_list.append('0400')
    elif one >= 4000 and one < 4250:
        weight_code_list.append('0425')
    elif one >= 4250 and one < 4500:
        weight_code_list.append('0450')
    elif one >= 4500 and one < 4750:
        weight_code_list.append('0475')
    elif one >= 4750 and one < 5000:
        weight_code_list.append('0500')
    elif one >= 5000 and one < 5250:
        weight_code_list.append('0525')
    elif one >= 5250 and one < 5500:
        weight_code_list.append('0550')
    elif one >= 5500 and one < 5750:
        weight_code_list.append('0575')
    elif one >= 5750 and one < 6000:
        weight_code_list.append('0600')
    elif one >= 6000 and one < 6250:
        weight_code_list.append('0625')
    elif one >= 6250 and one < 6500:
        weight_code_list.append('0650')
    elif one >= 6500 and one < 6750:
        weight_code_list.append('0675')
    elif one >= 6750 and one < 7000:
        weight_code_list.append('0700')
    elif one >= 7000 and one < 7250:
        weight_code_list.append('0725')
    elif one >= 7250 and one < 7500:
        weight_code_list.append('0750')
    elif one >= 7500 and one < 7750:
        weight_code_list.append('0775')
    elif one >= 7750 and one < 8000:
        weight_code_list.append('0800')
    elif one >= 8000 and one < 8250:
        weight_code_list.append('0825')
    elif one >= 8250 and one < 8500:
        weight_code_list.append('0850')
    elif one >= 8500 and one < 8750:
        weight_code_list.append('0875')
    elif one >= 8750 and one < 9000:
        weight_code_list.append('0900')
    elif one >= 9000 and one < 9250:
        weight_code_list.append('0925')
    elif one >= 9250 and one < 9500:
        weight_code_list.append('0950')
    elif one >= 9500 and one < 9750:
        weight_code_list.append('0975')
    elif one >= 9750 and one < 10000:
        weight_code_list.append('1000')
    elif one >= 10000 and one < 10250:
        weight_code_list.append('1025')
    elif one >= 10250 and one < 10500:
        weight_code_list.append('1050')
    elif one >= 10500 and one < 10750:
        weight_code_list.append('1075')
    elif one >= 10750 and one < 11000:
        weight_code_list.append('1100')
    elif one >= 11000 and one < 11250:
        weight_code_list.append('1125')
    elif one >= 11250 and one < 11500:
        weight_code_list.append('1150')
    elif one >= 11500 and one < 11750:
        weight_code_list.append('1175')
    elif one >= 11750 and one < 12000:
        weight_code_list.append('1200')
    elif one >= 12000 and one < 12250:
        weight_code_list.append('1225')
    elif one >= 12250 and one < 12500:
        weight_code_list.append('1250')
    elif one >= 12500 and one < 12750:
        weight_code_list.append('1275')
    elif one >= 12750 and one < 13000:
        weight_code_list.append('1300')
    elif one >= 13000 and one < 13250:
        weight_code_list.append('1325')
    elif one >= 13250 and one < 13500:
        weight_code_list.append('1350')
    elif one >= 13500 and one < 13750:
        weight_code_list.append('1375')
    elif one >= 13750 and one < 14000:
        weight_code_list.append('1400')
    elif one >= 14000 and one < 14250:
        weight_code_list.append('1425')
    elif one >= 14250 and one < 14500:
        weight_code_list.append('1450')
    elif one >= 14500 and one < 14750:
        weight_code_list.append('1475')
    elif one >= 14750 and one < 15000:
        weight_code_list.append('1500')
    elif one >= 15000 and one < 15250:
        weight_code_list.append('1525')
    elif one >= 15250 and one < 15500:
        weight_code_list.append('1550')
    elif one >= 15500 and one < 15750:
        weight_code_list.append('1575')
    elif one >= 15750 and one < 16000:
        weight_code_list.append('1600')
    elif one >= 16000 and one < 16250:
        weight_code_list.append('1625')
    elif one >= 16250 and one < 16500:
        weight_code_list.append('1650')
    elif one >= 16500 and one < 16750:
        weight_code_list.append('1675')
    elif one >= 16750 and one < 17000:
        weight_code_list.append('1700')
    elif one >= 17000 and one < 17250:
        weight_code_list.append('1725')
    elif one >= 17250 and one < 17500:
        weight_code_list.append('1750')
    elif one >= 17500 and one < 17750:
        weight_code_list.append('1775')
    elif one >= 17750 and one < 18000:
        weight_code_list.append('1800')
    elif one >= 18000 and one < 18250:
        weight_code_list.append('1825')
    elif one >= 18250 and one < 18500:
        weight_code_list.append('1850')
    elif one >= 18500 and one < 18750:
        weight_code_list.append('1875')
    elif one >= 18750 and one < 19000:
        weight_code_list.append('1900')
    elif one >= 19000 and one < 19250:
        weight_code_list.append('1925')
    elif one >= 19250 and one < 19500:
        weight_code_list.append('1950')
    elif one >= 19500 and one < 19750:
        weight_code_list.append('1975')
    elif one >= 19750 and one < 20000:
        weight_code_list.append('2000')
    elif one >= 20000 and one < 20250:
        weight_code_list.append('2025')
    elif one >= 20250 and one < 20500:
        weight_code_list.append('2050')
    elif one >= 20500 and one < 20750:
        weight_code_list.append('2075')
    elif one >= 20750 and one < 21000:
        weight_code_list.append('2100')
    elif one >= 21000 and one < 21250:
        weight_code_list.append('2125')
    elif one >= 21250 and one < 21500:
        weight_code_list.append('2150')
    elif one >= 21500 and one < 21750:
        weight_code_list.append('2175')
    elif one >= 21750 and one < 22000:
        weight_code_list.append('2200')
    elif one >= 22000 and one < 22250:
        weight_code_list.append('2225')
    elif one >= 22250 and one < 22500:
        weight_code_list.append('2250')
    elif one >= 22500 and one < 22750:
        weight_code_list.append('2275')
    elif one >= 22750 and one < 23000:
        weight_code_list.append('2300')
    elif one >= 23000 and one < 23250:
        weight_code_list.append('2325')
    elif one >= 23250 and one < 23500:
        weight_code_list.append('2350')
    elif one >= 23500 and one < 23750:
        weight_code_list.append('2375')
    elif one >= 23750 and one < 24000:
        weight_code_list.append('2400')
    elif one >= 24000 and one < 24250:
        weight_code_list.append('2425')
    elif one >= 24250 and one < 24500:
        weight_code_list.append('2450')
    elif one >= 24500 and one < 24750:
        weight_code_list.append('2475')
    elif one >= 24750 and one < 25000:
        weight_code_list.append('2500')
    elif one >= 25000 and one < 25250:
        weight_code_list.append('2525')
    elif one >= 25250 and one < 25500:
        weight_code_list.append('2550')
    elif one >= 25500 and one < 25750:
        weight_code_list.append('2575')
    elif one >= 25750 and one < 26000:
        weight_code_list.append('2600')
    elif one >= 26000 and one < 26250:
        weight_code_list.append('2625')
    elif one >= 26250 and one < 26500:
        weight_code_list.append('2650')
    elif one >= 26500 and one < 26750:
        weight_code_list.append('2675')
    elif one >= 26750 and one < 27000:
        weight_code_list.append('2700')
    elif one >= 27000 and one < 27250:
        weight_code_list.append('2725')
    elif one >= 27250 and one < 27500:
        weight_code_list.append('2750')
    elif one >= 27500 and one < 27750:
        weight_code_list.append('2775')
    elif one >= 27750 and one < 28000:
        weight_code_list.append('2800')
    elif one >= 28000 and one < 28250:
        weight_code_list.append('2825')
    elif one >= 28250 and one < 28500:
        weight_code_list.append('2850')
    elif one >= 28500 and one < 28750:
        weight_code_list.append('2875')
    elif one >= 28750 and one < 29000:
        weight_code_list.append('2900')
    elif one >= 29000 and one < 29250:
        weight_code_list.append('2925')
    elif one >= 29250 and one < 29500:
        weight_code_list.append('2950')
    elif one >= 29500 and one < 29750:
        weight_code_list.append('2975')
    elif one >= 29750 and one < 30000:
        weight_code_list.append('3000')
    elif one >= 30000 and one < 30250:
        weight_code_list.append('3025')
    elif one >= 30250 and one < 30500:
        weight_code_list.append('3050')
    elif one >= 30500 and one < 30750:
        weight_code_list.append('3075')
    elif one >= 30750 and one < 31000:
        weight_code_list.append('3100')
    elif one >= 31000 and one < 31250:
        weight_code_list.append('3125')
    elif one >= 31250 and one < 31500:
        weight_code_list.append('3150')
    elif one >= 31500 and one < 31750:
        weight_code_list.append('3175')
    elif one >= 31750 and one < 32000:
        weight_code_list.append('3200')
    elif one >= 32000 and one < 32250:
        weight_code_list.append('3225')
    elif one >= 32250 and one < 32500:
        weight_code_list.append('3250')
    elif one >= 32500 and one < 32750:
        weight_code_list.append('3275')
    elif one >= 32750 and one < 33000:
        weight_code_list.append('3300')
    elif one >= 33000 and one < 33250:
        weight_code_list.append('3325')
    elif one >= 33250 and one < 33500:
        weight_code_list.append('3350')
    elif one >= 33500 and one < 33750:
        weight_code_list.append('3375')
    elif one >= 33750 and one < 34000:
        weight_code_list.append('3400')
    elif one >= 34000 and one < 34250:
        weight_code_list.append('3425')
    elif one >= 34250 and one < 34500:
        weight_code_list.append('3450')
    elif one >= 34500 and one < 34750:
        weight_code_list.append('3475')
    elif one >= 34750 and one < 35000:
        weight_code_list.append('3500')
    elif one >= 35000 and one < 35250:
        weight_code_list.append('3525')
    elif one >= 35250 and one < 35500:
        weight_code_list.append('3550')
    elif one >= 35500 and one < 35750:
        weight_code_list.append('3575')
    elif one >= 35750 and one < 36000:
        weight_code_list.append('3600')
    elif one >= 36000 and one < 36250:
        weight_code_list.append('3625')
    elif one >= 36250 and one < 36500:
        weight_code_list.append('3650')
    elif one >= 36500 and one < 36750:
        weight_code_list.append('3675')
    elif one >= 36750 and one < 37000:
        weight_code_list.append('3700')
    elif one >= 37000 and one < 37250:
        weight_code_list.append('3725')
    elif one >= 37250 and one < 37500:
        weight_code_list.append('3750')
    elif one >= 37500 and one < 37750:
        weight_code_list.append('3775')
    elif one >= 37750 and one < 38000:
        weight_code_list.append('3800')
    elif one >= 38000 and one < 38250:
        weight_code_list.append('3825')
    elif one >= 38250 and one < 38500:
        weight_code_list.append('3850')
    elif one >= 38500 and one < 38750:
        weight_code_list.append('3875')
    elif one >= 38750 and one < 39000:
        weight_code_list.append('3900')
    elif one >= 39000 and one < 39250:
        weight_code_list.append('3925')
    elif one >= 39250 and one < 39500:
        weight_code_list.append('3950')
    elif one >= 39500 and one < 39750:
        weight_code_list.append('3975')
    elif one >= 39750 and one < 40000:
        weight_code_list.append('4000')
    elif one >= 40000 and one < 40250:
        weight_code_list.append('4025')
    elif one >= 40250 and one < 40500:
        weight_code_list.append('4050')
    elif one >= 40500 and one < 40750:
        weight_code_list.append('4075')
    elif one >= 40750 and one < 41000:
        weight_code_list.append('4100')
    elif one >= 41000 and one < 41250:
        weight_code_list.append('4125')
    elif one >= 41250 and one < 41500:
        weight_code_list.append('4050')
    elif one >= 41500 and one < 41750:
        weight_code_list.append('4175')
    elif one >= 41750 and one < 42000:
        weight_code_list.append('4200')
    elif one >= 42000 and one < 42250:
        weight_code_list.append('4225')
    elif one >= 42250 and one < 42500:
        weight_code_list.append('4250')
    elif one >= 42500 and one < 42750:
        weight_code_list.append('4275')
    elif one >= 42750 and one < 43000:
        weight_code_list.append('4300')
    elif one >= 43000 and one < 43250:
        weight_code_list.append('4325')
    elif one >= 43250 and one < 43500:
        weight_code_list.append('4350')
    elif one >= 43500 and one < 43750:
        weight_code_list.append('4375')
    elif one >= 43750 and one < 44000:
        weight_code_list.append('4400')
    elif one >= 44000 and one < 44250:
        weight_code_list.append('4425')
    elif one >= 44250 and one < 44500:
        weight_code_list.append('4450')
    elif one >= 44500 and one < 44750:
        weight_code_list.append('4475')
    elif one >= 44750 and one < 45000:
        weight_code_list.append('4500')
    elif one >= 45000 and one < 45250:
        weight_code_list.append('4525')
    elif one >= 45250 and one < 45500:
        weight_code_list.append('4550')
    elif one >= 45500 and one < 45750:
        weight_code_list.append('4575')
    elif one >= 45750 and one < 46000:
        weight_code_list.append('4600')
    elif one >= 46000 and one < 46250:
        weight_code_list.append('4625')
    elif one >= 46250 and one < 46500:
        weight_code_list.append('4650')
    elif one >= 46500 and one < 46750:
        weight_code_list.append('4675')
    elif one >= 46750 and one < 47000:
        weight_code_list.append('4700')
    elif one >= 47000 and one < 47250:
        weight_code_list.append('4725')
    elif one >= 47250 and one < 47500:
        weight_code_list.append('4750')
    elif one >= 47500 and one < 47750:
        weight_code_list.append('4775')
    elif one >= 47750 and one < 48000:
        weight_code_list.append('4800')
    elif one >= 48000 and one < 48250:
        weight_code_list.append('4825')
    elif one >= 48250 and one < 48500:
        weight_code_list.append('4850')
    elif one >= 48500 and one < 48750:
        weight_code_list.append('4875')
    elif one >= 48750 and one < 49000:
        weight_code_list.append('4900')
    elif one >= 49000 and one < 49250:
        weight_code_list.append('4925')
    elif one >= 49250 and one < 49500:
        weight_code_list.append('4950')
    elif one >= 49500 and one < 49750:
        weight_code_list.append('4975')
    elif one >= 49750 and one < 50000:
        weight_code_list.append('5000')
    elif one >= 50000:
        weight_code_list.append('9999')

1528557it [00:02, 537521.71it/s]


In [187]:
len(weight_code_list)

1528557

In [188]:
gas_df.shape

(1528557, 47)

In [189]:
gas_df['총중량_코드'] = weight_code_list

In [190]:
n = 250
f'{n:0>4}'

'0250'

In [191]:
# 오류 : 9000개만 처리됨
# weight_code_list = []
# n1 = 250
# for i, one in tqdm(enumerate(df5['총중량'])):
#     for i in range(200):
#         n1 += 250 * i
#         n2 = n1 + 250
#         s = f'{n2:0>4}'
#         if one < 250:
#             weight_code_list.append('0025')
#         elif n1 <= one < n2:
#             weight_code_list.append(s)
#         elif one >= 50000:
#             weight_code_list.append('9999')
# len(weight_code_list)

### 배기량으로 분류 현황

In [192]:
gas_df['배기량'].value_counts()

2497.0    412725
1991.0    245077
2902.0    139727
2959.0     75052
1995.0     68704
           ...  
2687.0         1
5407.0         1
2605.0         1
1747.0         1
2286.0         1
Name: 배기량, Length: 627, dtype: int64

In [193]:
gas_df['배기량'].isnull().sum()

4

In [194]:
gas_df['배기량'] = gas_df['배기량'].fillna(-1)

In [195]:
gas_volume_list = []
for i, one in tqdm(enumerate(gas_df['배기량'])):
    if one == -1:
        gas_volume_list.append('미기입')
    elif one < 800:
        gas_volume_list.append('0.8L 미만')
    elif one >= 800 and one < 1000:
        gas_volume_list.append('0.8L 이상 1.0L 미만')
    elif one >= 1000 and one < 1600:
        gas_volume_list.append('1.0L 이상 1.6L 미만')
    elif one >= 1600 and one < 2000:
        gas_volume_list.append('1.6L 이상 2.0L 미만')
    elif one >= 2000 and one < 3000:
        gas_volume_list.append('2.0L 이상 3.0L 미만')
    elif one >= 3000 and one < 4000:
        gas_volume_list.append('3.0L 이상 4.0L 미만')
    elif one >= 4000 and one < 5000:
        gas_volume_list.append('4.0L 이상 5.0L 미만')
    elif one >= 5000 and one < 6000:
        gas_volume_list.append('5.0L 이상 6.0L 미만')
    elif one >= 6000 and one < 8000:
        gas_volume_list.append('6.0L 이상 8.0L 미만')
    elif one >= 8000 and one < 10000:
        gas_volume_list.append('8.0L 이상 10.0L 미만')
    elif one >= 10000:
        gas_volume_list.append('10.0L 이상')

1528557it [00:01, 993921.31it/s] 


In [196]:
len(gas_volume_list)

1528557

In [197]:
gas_df['배기량_분류'] = gas_volume_list

In [198]:
total_gas_volume_df = gas_df.groupby(['배기량_분류'])['차대번호'].count().reset_index()
total_gas_volume_df

,배기량_분류,차대번호
0,0.8L 미만,28673
1,0.8L 이상 1.0L 미만,171
2,1.0L 이상 1.6L 미만,165909
3,1.6L 이상 2.0L 미만,510876
4,10.0L 이상,16847
5,2.0L 이상 3.0L 미만,755436
6,3.0L 이상 4.0L 미만,31434
7,4.0L 이상 5.0L 미만,1340
8,5.0L 이상 6.0L 미만,15493
9,6.0L 이상 8.0L 미만,711


In [199]:
total_gas_volume_df.to_excel('analysis/[G4][배기량]분류_통계.xlsx', encoding='utf-8')

#### 배기량 code화

In [200]:
gas_volume_code_list = []
for i, one in tqdm(enumerate(gas_df['배기량'])):
    if one < 800:
        gas_volume_code_list.append('008')
    elif one >= 800 and one < 1000:
        gas_volume_code_list.append('010')
    elif one >= 1000 and one < 1600:
        gas_volume_code_list.append('016')
    elif one >= 1600 and one < 2000:
        gas_volume_code_list.append('020')
    elif one >= 2000 and one < 2500:
        gas_volume_code_list.append('025')
    elif one >= 2500 and one < 3000:
        gas_volume_code_list.append('030')
    elif one >= 3000 and one < 3500:
        gas_volume_code_list.append('035')
    elif one >= 3500 and one < 4000:
        gas_volume_code_list.append('040')
    elif one >= 4000 and one < 4500:
        gas_volume_code_list.append('045')
    elif one >= 4500 and one < 5000:
        gas_volume_code_list.append('050')
    elif one >= 5000 and one < 5500:
        gas_volume_code_list.append('055')
    elif one >= 5500 and one < 6000:
        gas_volume_code_list.append('060')
    elif one >= 6000 and one < 6500:
        gas_volume_code_list.append('065')
    elif one >= 6500 and one < 7000:
        gas_volume_code_list.append('070')
    elif one >= 7000 and one < 7500:
        gas_volume_code_list.append('075')
    elif one >= 7500 and one < 8000:
        gas_volume_code_list.append('080')
    elif one >= 8000 and one < 8500:
        gas_volume_code_list.append('085')
    elif one >= 8500 and one < 9000:
        gas_volume_code_list.append('090')
    elif one >= 9000 and one < 9500:
        gas_volume_code_list.append('095')
    elif one >= 9500 and one < 10000:
        gas_volume_code_list.append('100')
    elif one >= 10000 and one < 10500:
        gas_volume_code_list.append('105')
    elif one >= 10500 and one < 11000:
        gas_volume_code_list.append('110')
    elif one >= 11000 and one < 11500:
        gas_volume_code_list.append('115')
    elif one >= 11500 and one < 12000:
        gas_volume_code_list.append('120')
    elif one >= 12000 and one < 12500:
        gas_volume_code_list.append('125')
    elif one >= 12500 and one < 13000:
        gas_volume_code_list.append('130')
    elif one >= 13000 and one < 13500:
        gas_volume_code_list.append('135')
    elif one >= 13500 and one < 14000:
        gas_volume_code_list.append('140')
    elif one >= 14000 and one < 14500:
        gas_volume_code_list.append('145')
    elif one >= 14500 and one < 15000:
        gas_volume_code_list.append('150')
    elif one >= 15000 and one < 15500:
        gas_volume_code_list.append('155')
    elif one >= 15500 and one < 16000:
        gas_volume_code_list.append('160')
    elif one >= 16000 and one < 16500:
        gas_volume_code_list.append('165')
    elif one >= 16500 and one < 17000:
        gas_volume_code_list.append('170')
    elif one >= 17000 and one < 18000:
        gas_volume_code_list.append('180')
    elif one >= 18000 and one < 18500:
        gas_volume_code_list.append('185')
    elif one >= 18500 and one < 19000:
        gas_volume_code_list.append('190')
    elif one >= 19000 and one < 19500:
        gas_volume_code_list.append('195')
    elif one >= 19500 and one < 20000:
        gas_volume_code_list.append('200')
    elif one >= 20000 and one < 20500:
        gas_volume_code_list.append('205')
    elif one >= 20500 and one < 21000:
        gas_volume_code_list.append('210')
    elif one >= 21000 and one < 21500:
        gas_volume_code_list.append('215')
    elif one >= 21500 and one < 22000:
        gas_volume_code_list.append('220')
    elif one >= 22000 and one < 22500:
        gas_volume_code_list.append('225')
    elif one >= 22500 and one < 23000:
        gas_volume_code_list.append('230')
    elif one >= 23000 and one < 23500:
        gas_volume_code_list.append('235')
    elif one >= 23500 and one < 24000:
        gas_volume_code_list.append('240')
    elif one >= 24000 and one < 24500:
        gas_volume_code_list.append('245')
    elif one >= 24500 and one < 25000:
        gas_volume_code_list.append('250')
    elif one >= 25000 and one < 25500:
        gas_volume_code_list.append('255')
    elif one >= 25500 and one < 26000:
        gas_volume_code_list.append('260')
    elif one >= 26000 and one < 26500:
        gas_volume_code_list.append('265')
    elif one >= 26500 and one < 27000:
        gas_volume_code_list.append('270')
    elif one >= 27000 and one < 27500:
        gas_volume_code_list.append('275')
    elif one >= 27500 and one < 28000:
        gas_volume_code_list.append('280')
    elif one >= 28000 and one < 28500:
        gas_volume_code_list.append('285')
    elif one >= 28500 and one < 29000:
        gas_volume_code_list.append('290')
    elif one >= 29000 and one < 29500:
        gas_volume_code_list.append('295')
    elif one >= 29500 and one < 30000:
        gas_volume_code_list.append('300')

1528557it [00:01, 968941.31it/s] 


In [201]:
len(gas_volume_code_list)

1528557

In [202]:
gas_df['배기량_코드'] = gas_volume_code_list

In [203]:
gas_df['배출가스인증번호'] = gas_df['배출가스인증번호'].str.replace(' ', '')

### 배인 코드화

In [204]:
gas_df['배출가스인증번호'].unique()

array([nan, '7MY-HD-14-68', '7MY-HD-24-31', '7MY-HD-24-09',
       '7MY-HD-24-28', '7MY-KM-14-65', '6MY-SS-0M-24', '7MY-HD-13-58',
       '7MY-KM-14-63', '7MY-HD-14-47', '7MY-HD-14-46', '7MY-DB-24-42',
       '8MY-KM-13-19', '7MY-HD-14-57', '7MY-KM-14-64', '9MY-KM-14-41',
       '7MY-HD-24-30', 'FMY-VTK-14-1', '5MY-HD-24-59', '6MY-HD-23-55',
       '6MY-KM-14-61', '7MY-DW-13-07', '7MY-HD-24-29', '5MY-AD-14-18',
       'FMY-VTK-14-4', '6MY-HD-24-12', '7MY-SY-14-48', '6MY-HD-24-100',
       '7MY-HD-14-45', 'FMY-VTK-14-2', 'FMY-VTK-14-3', 'RMY-HD-20',
       'VMY-HD-30', 'WMY-HD-32', '5MY-AD-14-12', '9MY-HD-14-21',
       '5MY-AD-14-08', '7MY-SY-14-60', '7MY-SM-14-36', '6MY-HD-14-97',
       '7MY-HD-24-08', '6MY-KM-24-81', 'CMY-VTK-14-01', 'CMY-VTK-14-02',
       'KMY-HD-03', '6MY-HD-14-64', 'MMY-HD-11', '5MY-SY-23-58',
       '6MY-HD-14-63', 'SMY-KM-32', '6MY-KM-24-06', '7MY-KM-14-52',
       '8MY-HT-0M-24', '6MY-HD-24-44', '7MY-SY-14-34', '5MY-HB-24-02',
       'BMY-PPE-14-29', '6MY-DW-

In [205]:
temp = gas_df['배출가스인증번호'].value_counts().reset_index().sort_values('index')['index'].reset_index(drop=True)
temp

0         3MY-FK-08
1         4MY-AD-20
2         4MY-DC-33
3         4MY-PA-06
4      5MY-AD-13-06
           ...     
295       XMY-HD-47
296       XMY-HD-48
297       XMY-HD-55
298       XMY-KM-47
299       YKM-KM-12
Name: index, Length: 300, dtype: object

In [206]:
temp.to_frame()

,index
0,3MY-FK-08
1,4MY-AD-20
2,4MY-DC-33
3,4MY-PA-06
4,5MY-AD-13-06
...,...
295,XMY-HD-47
296,XMY-HD-48
297,XMY-HD-55
298,XMY-KM-47


In [207]:
gas_idx = []
for i in range(1, 301):
    s = f'EN{i:0>3}'
    gas_idx.append(s)
len(gas_idx)

300

In [208]:
gas_idx_df = pd.DataFrame({'배출가스인증번호_코드':gas_idx})
gas_idx_df

,배출가스인증번호_코드
0,EN001
1,EN002
2,EN003
3,EN004
4,EN005
...,...
295,EN296
296,EN297
297,EN298
298,EN299


In [209]:
temp2 = pd.concat([temp, gas_idx_df], axis=1, ignore_index=True)
temp2

,0,1
0,3MY-FK-08,EN001
1,4MY-AD-20,EN002
2,4MY-DC-33,EN003
3,4MY-PA-06,EN004
4,5MY-AD-13-06,EN005
...,...,...
295,XMY-HD-47,EN296
296,XMY-HD-48,EN297
297,XMY-HD-55,EN298
298,XMY-KM-47,EN299


In [210]:
temp2.columns = ['배출가스인증번호', '배출가스인증번호_코드']
temp2

,배출가스인증번호,배출가스인증번호_코드
0,3MY-FK-08,EN001
1,4MY-AD-20,EN002
2,4MY-DC-33,EN003
3,4MY-PA-06,EN004
4,5MY-AD-13-06,EN005
...,...,...
295,XMY-HD-47,EN296
296,XMY-HD-48,EN297
297,XMY-HD-55,EN298
298,XMY-KM-47,EN299


In [211]:
temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   배출가스인증번호     300 non-null    object
 1   배출가스인증번호_코드  300 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB


In [212]:
gas_df.shape

(1528557, 50)

In [213]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN',
       '총중량_분류', '총중량_코드', '배기량_분류', '배기량_코드'],
      dtype='object')

In [214]:
gas_df = gas_df.merge(temp2, on='배출가스인증번호', how='left')
gas_df.shape

(1528557, 51)

In [215]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보', '차종분류',
       '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN',
       '총중량_분류', '총중량_코드', '배기량_분류', '배기량_코드', '배출가스인증번호_코드'],
      dtype='object')

## 차대번호로 분류

### wmi 코드로 분류

In [216]:
wmi = pd.read_excel('data/20190730_WMI CODE.xlsx', sheet_name='WMI CODE')
wmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   WMI            690 non-null    object 
 1   MF CODE        690 non-null    object 
 2   Identifier     690 non-null    object 
 3   Manufacturer   690 non-null    object 
 4   Country        688 non-null    object 
 5   Vehicle Types  249 non-null    object 
 6   Description    54 non-null     object 
 7   참고             0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 43.2+ KB


In [217]:
df6 = gas_df.copy()

In [218]:
df6['차대번호'].head()

0    KMCLK18KP8C035195
1    KMFLA18KP8C031274
2    KLACA26RD8B226753
3    KMCGK17HP8C086937
4    KMCYSS7JP7U084697
Name: 차대번호, dtype: object

In [219]:
df6['wmi'] = df6['차대번호'].str[:3]

In [220]:
df6['wmi'].head()

0    KMC
1    KMF
2    KLA
3    KMC
4    KMC
Name: wmi, dtype: object

In [221]:
df6 = df6.merge(wmi, left_on='wmi', right_on='WMI', how='left')
df6[['차대번호', 'wmi', 'WMI']].head()

,차대번호,wmi,WMI
0,KMCLK18KP8C035195,KMC,NaN
1,KMFLA18KP8C031274,KMF,KMF
2,KLACA26RD8B226753,KLA,KLA
3,KMCGK17HP8C086937,KMC,NaN
4,KMCYSS7JP7U084697,KMC,NaN


In [222]:
df6.shape

(1528557, 60)

In [223]:
df6['WMI'].isnull().sum()

75232

In [224]:
df6['Manufacturer'].isnull().sum()

75232

In [225]:
wmi.columns

Index(['WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country',
       'Vehicle Types', 'Description', '참고'],
      dtype='object')

In [226]:
df6[['WMI', 'MF CODE', 'Identifier', 'Manufacturer', 'Country', 'Vehicle Types', 'Description', '참고']].head()

,WMI,MF CODE,Identifier,Manufacturer,Country,Vehicle Types,Description,참고
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMF,HYUN,Hyundai,Hyundai,Korea,Commercail vehicle,NaN,NaN
2,KLA,GM D,GM Daewoo,Daewoo General Motors South Korea,Korea,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [227]:
df6['Identifier'].isnull().sum()

75232

In [228]:
df6['Manufacturer'] = df6['Manufacturer'].fillna(-1)
df6['Identifier'] = df6['Identifier'].fillna(-1)

In [229]:
manu_df = df6.groupby('Identifier')['차대번호'].count().reset_index()
manu_df

,Identifier,차대번호
0,-1,75232
1,Alfa Romeo,1
2,Audi,2107
3,Austin-Rover,83
4,BMW,2209
...,...,...
71,Toyota,140
72,Toyota,35
73,Volkswagen,5924
74,Volvo,6667


In [230]:
manu_df = manu_df.sort_values('차대번호', ascending=False).reset_index(drop=True)
manu_df

,Identifier,차대번호
0,Hyundai,887146
1,Kia,358326
2,GM Daewoo,116233
3,-1,75232
4,SsangYong,31200
...,...,...
71,Neoplan,1
72,Matra,1
73,Daimler,1
74,Kenworth,1


In [231]:
# 제작사(Manufacturer)별 차대번호 통계
manu_df.to_excel('analysis/[G4][차대번호]제작사별_통계.xlsx', encoding='utf-8')

### 6\. [e]차대번호

In [232]:
# - 오류
#     - 1\. null 값 : 0
#     - 2\. 정상(17자리, 영문 포함) : 1525955
#     - 3\. 17자리, 영문X : 1
#     - 4\. 17자리X, 영문 : 57
#     - 5\. 17자리X, 영문X : 2573

In [233]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528557 entries, 0 to 1528556
Data columns (total 60 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   차량관리번호         1528557 non-null  object 
 1   차량등록번호         1528557 non-null  object 
 2   본거지법정동코드       1528557 non-null  int64  
 3   소유자구분          1528557 non-null  object 
 4   차종_등록정보        1528555 non-null  object 
 5   용도             1528557 non-null  object 
 6   차대번호           1528557 non-null  object 
 7   최초등록일자         1528557 non-null  int64  
 8   차량연식           1528557 non-null  int64  
 9   차량제작일자         1358915 non-null  float64
 10  검사유효일          1528545 non-null  float64
 11  제원관리번호         1528557 non-null  object 
 12  배출가스인증번호       1405554 non-null  object 
 13  배출가스등급         1528557 non-null  int64  
 14  저공해조치종류        195 non-null      object 
 15  저공해조치구조변경YN    1528557 non-null  object 
 16  취득일자           1435498 non-null  object 
 17  취득금액    

In [234]:
df6['차대번호'].isnull().sum()

0

In [235]:
len('KMFZCY7JBCU784032')

17

#### 차대번호 정상 확인

In [236]:
# 17자리, 영문 -> 정상 : 1525955
count = 0
p61 = re.compile('[a-zA-Z]+')
idx61 = []
for i, one in tqdm(enumerate(df6.iloc[:, 6].to_numpy())):
    if len(str(one)) == 17 and p61.search(str(one)):
        idx61.append(i)
        count += 1
count

1528557it [00:02, 619928.06it/s]


1525926

In [237]:
# 정상에서 "I, O, Q" 포함한 값 : 1018
temp61 = df6.iloc[idx61, 6].to_numpy()
idx61_1 = []
for i, one in tqdm(enumerate(temp61)):
    if ('I' in one) or ('O' in one) or ('Q' in one):
        idx61_1.append(idx61[i])
len(idx61_1)

1525926it [00:00, 1754444.44it/s]


1018

In [238]:
df6.iloc[idx61_1, 6][:5]

1142      WPOZZZ96ZLS402952
117803    WPOZZZ96ZKS400603
330303    YV2ABSOC4AA695671
346135    YV2ABSOC1AA694168
346256    YV2ABSOC2AA693157
Name: 차대번호, dtype: object

In [239]:
# 17자리, 영문X  => 숫자만 : 1
count = 0
p62 = re.compile('[a-zA-Z]+')
idx62 = []
for i, one in tqdm(enumerate(df6.iloc[:, 6].to_numpy())):
    n = 0
    for j in str(one):
        if p62.search(j):
            n += 1
    if len(str(one)) == 17 and n == 0:
        idx62.append(i)
        count += 1
count

1528557it [00:12, 123067.59it/s]


1

In [240]:
df6.iloc[:, 6].reindex(idx62)

1471703    01901321371111111
Name: 차대번호, dtype: object

In [241]:
# 17자리 X, 영문 : 57
count = 0
p63 = re.compile('[a-zA-Z]+')
idx63 = []
for i, one in tqdm(enumerate(df6.iloc[:, 6].to_numpy())):
    if len(str(one)) != 17 and p63.search(one):
        idx63.append(i)
        count += 1
count

1528557it [00:01, 1365514.82it/s]


57

In [242]:
df6.iloc[:, 6].reindex(idx63)[:5]

352051    KMFWBX7ABU267970
362014    KMFZCZ7JAB640917
461606    KNCSJX4AAK452185
641800    KMFZC7JP9U519465
731532    KMFZC7JP9U481571
Name: 차대번호, dtype: object

In [243]:
# 17자리 X, 영문X : 2573
count = 0
p64 = re.compile('[a-zA-Z]+')
idx64 = []
for i, one in tqdm(enumerate(df6.iloc[:, 6].to_numpy())):
    n = 0
    for j in one:
        if p64.search(j):
            n += 1
    if len(str(one)) != 17 and n == 0:
        idx64.append(i)
        count += 1
count

1528557it [00:12, 120512.92it/s]


2573

In [244]:
df6.iloc[:, 6].reindex(idx64)[:5]

1144821    289711309283
1161719    119811229792
1170369    119811328945
1170707    119411329496
1170967    119311230058
Name: 차대번호, dtype: object

#### 차대번호 길이, IOQ포함 여부 통계

In [245]:
# 차대번호 길이 종류 확인
len_vin = []
for i, one in tqdm(enumerate(df6.iloc[:, 6].to_numpy())):
    len_vin.append(len(str(one)))
set(len_vin)

1528557it [00:01, 1250873.15it/s]


{10, 11, 12, 13, 14, 15, 16, 17}

In [246]:
df6['VIN_length'] = [ len(one) for one in df6['차대번호'] ]

In [247]:
df6['VIN_length'].unique()

array([17, 16, 12, 15, 11, 13, 14, 10], dtype=int64)

In [248]:
# IOQ 포함 여부
IOQ = []
for one in df6['차대번호']:
    if ('I' in one) or ('O' in one) or ('Q' in one):
        IOQ.append('Y')
    else:
        IOQ.append('N')
df6['IOQ'] = IOQ

In [249]:
df6['IOQ'].unique()

array(['N', 'Y'], dtype=object)

In [250]:
# 알파벳 포함
included_alpha = []
count = 0
p65 = re.compile('[a-zA-Z]+')
for i, one in tqdm(enumerate(df6.iloc[:, 6].to_numpy())):
    n = 0
    for j in one:
        if p65.search(j):
            n += 1
    if n == 0:
        included_alpha.append('N')
        count += 1
    else:
        included_alpha.append('Y')
count

1528557it [00:12, 126412.99it/s]


2574

In [251]:
df6['alpha'] = included_alpha

In [252]:
df6.groupby(['VIN_length', 'alpha', 'IOQ'])['차대번호'].count()

VIN_length  alpha  IOQ
10          N      N            1
            Y      N            3
11          N      N          209
            Y      N            7
12          N      N         2132
            Y      N            2
13          N      N          154
            Y      N            3
14          N      N           22
15          N      N           38
            Y      N            2
                   Y            1
16          N      N           17
            Y      N           39
17          N      N            1
            Y      N      1524908
                   Y         1018
Name: 차대번호, dtype: int64

In [253]:
df6.groupby(['VIN_length', 'IOQ'])['차대번호'].count()

VIN_length  IOQ
10          N            4
11          N          216
12          N         2134
13          N          157
14          N           22
15          N           40
            Y            1
16          N           56
17          N      1524909
            Y         1018
Name: 차대번호, dtype: int64

In [254]:
V_I_dict = df6.groupby(['VIN_length', 'IOQ'])['차대번호'].groups

In [255]:
V_I_dict.keys()

dict_keys([(10, 'N'), (11, 'N'), (12, 'N'), (13, 'N'), (14, 'N'), (15, 'N'), (15, 'Y'), (16, 'N'), (17, 'N'), (17, 'Y')])

In [256]:
g = df6.groupby(['VIN_length', 'IOQ'], as_index=False)['차대번호'].count()
g

,VIN_length,IOQ,차대번호
0,10,N,4
1,11,N,216
2,12,N,2134
3,13,N,157
4,14,N,22
5,15,N,40
6,15,Y,1
7,16,N,56
8,17,N,1524909
9,17,Y,1018


In [257]:
V_I_df = g.sort_values(['VIN_length', 'IOQ'], ascending=[True, True])
V_I_df

,VIN_length,IOQ,차대번호
0,10,N,4
1,11,N,216
2,12,N,2134
3,13,N,157
4,14,N,22
5,15,N,40
6,15,Y,1
7,16,N,56
8,17,N,1524909
9,17,Y,1018


##### 차대번호 길이, IOQ포함 여부 별 예시

In [258]:
for v in range(10, 18):
    for i in ['N', 'Y']:
        try:
            idx = V_I_dict[(v, i)]
            print(f'[VIN_length : {v}, "IOQ"포함 : {i}]')
            print('예시)')
            print(df6.iloc[idx[:3], 6])
            print()
        except:
            pass

[VIN_length : 10, "IOQ"포함 : N]
예시)
1184741    FD3S112983
1191288    FD3S303919
1197803    EU13315046
Name: 차대번호, dtype: object

[VIN_length : 11, "IOQ"포함 : N]
예시)
1171917    H36A0602064
1179716    RPS13330414
1186002    DE2A0100210
Name: 차대번호, dtype: object

[VIN_length : 12, "IOQ"포함 : N]
예시)
1144821    289711309283
1161719    119811229792
1170369    119811328945
Name: 차대번호, dtype: object

[VIN_length : 13, "IOQ"포함 : N]
예시)
1176816    4197111236649
1179156    289711308190하
1180037    2893113157323
Name: 차대번호, dtype: object

[VIN_length : 14, "IOQ"포함 : N]
예시)
1182843    27911130156505
1183740    27931130152305
1349628    21891130315401
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : N]
예시)
1171406    11-97-113-29992
1171678    11-97-113-28686
1171805    41-96-113-35598
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : Y]
예시)
1528151    12851300018EORN
Name: 차대번호, dtype: object

[VIN_length : 16, "IOQ"포함 : N]
예시)
352051    KMFWBX7ABU267970
362014    KMFZCZ7JAB640917
461606   

In [259]:
df6.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T

VIN_length,10,11,12,13,14,15,16,17
연료,,,,,,,,
CNG(압축천연가스),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
LPG(액화석유가스),NaN,NaN,15.0,NaN,NaN,NaN,2.0,26401.0
경유,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1167595.0
기타연료,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
알코올,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
휘발유,4.0,216.0,2119.0,157.0,22.0,41.0,44.0,331928.0


In [260]:
# vin 길이별, 연료별 차대번호 개수 통계
df6.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]길이_통계.xlsx')

In [261]:
df6.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T

IOQ,N,Y
연료,,
CNG(압축천연가스),1.0,NaN
LPG(액화석유가스),26414.0,4.0
경유,1167405.0,200.0
기타연료,1.0,NaN
알코올,1.0,NaN
휘발유,333716.0,815.0


In [262]:
# vin IOQ 포함별, 연료별 차대번호 개수 통계
df6.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]IOQ포함여부_통계.xlsx')

#### 차대번호 10번째 자리 문자로 연식 확인

In [263]:
df6['차량연식'].unique()

array([2008, 2007, 2006, 2012, 2011, 2009, 2001, 2010, 2019, 2020, 1992,
       1994, 1996, 2018, 1991, 1993, 1989, 1990, 1995, 1997, 2013, 1998,
       1999, 1988, 2017, 2005, 2016, 2002, 2004, 2003, 2015, 1987, 2014,
       1986, 2000, 1981, 1978, 1973, 1984, 1901, 1985, 1980, 1979, 1983,
       1968], dtype=int64)

In [264]:
df6.groupby(['차량연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
차량연식,,,,,,
1901,NaN,NaN,NaN,NaN,NaN,1.0
1968,NaN,NaN,NaN,NaN,NaN,1.0
1973,NaN,NaN,NaN,NaN,NaN,1.0
1978,NaN,NaN,NaN,NaN,NaN,1.0
1979,NaN,NaN,NaN,NaN,NaN,3.0
1980,NaN,NaN,NaN,NaN,NaN,1.0
1981,NaN,NaN,NaN,NaN,NaN,1.0
1983,NaN,NaN,NaN,NaN,NaN,3.0
1984,NaN,NaN,NaN,NaN,NaN,3.0


In [265]:
# 차량연식별, 연료별 차대번호 개수 통계
df6.groupby(['차량연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]통계.xlsx')

In [266]:
df6['차대번호_10자리'] = df6['차대번호'].str[9]
df6['차대번호_10자리'].unique()

array(['8', '7', '6', 'C', 'B', '9', '1', 'A', 'K', 'L', 'N', 'R', 'T',
       'J', 'M', 'P', 'S', 'V', 'D', 'W', 'X', 'H', '5', 'G', '2', '4',
       '3', 'F', 'E', 'Y', 'U', '0', 'O', '-'], dtype=object)

In [267]:
vin_10_dict = {'A':1980, 'B':1981, 'C':1982, 'D':1983, 'E':1984, 'F':1985, 'G':1986, 'H':1987, 'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009}

In [268]:
df6['차대번호_10자리_기준연식'] = df6['차대번호_10자리'].replace(vin_10_dict)

In [269]:
df6['차대번호_10자리_기준연식'].unique()

array([2008, 2007, 2006, 1982, 1981, 2009, 2001, 1980, 1989, 1990, 1992,
       1994, 1996, 1988, 1991, 1993, 1995, 1997, 1983, 1998, 1999, 1987,
       2005, 1986, 2002, 2004, 2003, 1985, 1984, 2000, 'U', '0', 'O', '-'],
      dtype=object)

In [270]:
df6.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
차대번호_10자리_기준연식,,,,,,
1980,NaN,NaN,252705.0,NaN,NaN,2.0
1981,NaN,NaN,237573.0,NaN,NaN,5.0
1982,NaN,NaN,94135.0,NaN,NaN,54.0
1983,NaN,NaN,29.0,NaN,NaN,70.0
1984,NaN,NaN,13.0,NaN,NaN,1.0
1985,NaN,NaN,463.0,NaN,NaN,6.0
1986,NaN,NaN,482.0,NaN,NaN,12.0
1987,NaN,NaN,732.0,NaN,NaN,1299.0
1988,NaN,91.0,483.0,NaN,1.0,11652.0


In [271]:
# vin 10자리 문자기준별, 연료별 차대번호 통계
df6.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]차대번호10번째자리문자기준_통계.xlsx')

### 지역별 차량 대수

In [273]:
code = pd.read_excel('data/법정동코드_2022.10.14.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


In [274]:
code2 = code[code['폐지여부'] == '존재'].reset_index(drop=True)
code2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20550 entries, 0 to 20549
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     20550 non-null  int64  
 1   시도        20550 non-null  object 
 2   시구군       20533 non-null  object 
 3   읍면동       20240 non-null  object 
 4   리         15090 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      20550 non-null  object 
 7   대기관리권역    20550 non-null  object 
 8   대기관리권역YN  20544 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.4+ MB


## [로드]정기/정밀 검사 데이터 로드

In [253]:
# 4m 21.8s
per_ins = pd.read_csv('data/[정기검사]20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15180\1298806191.py:2: DtypeWarning: Columns (36,39,42,45,48,63) have mixed types. Specify dtype option on import or set low_memory=False.
  per_ins = pd.read_csv('data/[정기검사]20190701_20220630.csv', index_col=0)


In [254]:
per_ins.shape

(16535477, 63)

In [255]:
# 11m 11.3s
per_d_ins = pd.read_csv('data/[정밀검사]20190701_20220630.csv', index_col=0)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15180\1915627214.py:2: DtypeWarning: Columns (13,44,45,46,49,53,54,55,59,60,61,65,66,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  per_d_ins = pd.read_csv('data/[정밀검사]20190701_20220630.csv', index_col=0)


In [256]:
per_d_ins.shape

(22397343, 75)

In [281]:
per_ins.columns

Index(['검사소명', '정기검사일자', '검사종류', '검사접수번호', '통합접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일자', '제작일자',
       '검사전 유효기간 만료일', '관능검사판정', '관능부적합항목', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값', '산소값', '공기온도', '공기압력',
       '검사판정', '재검사기간', '검사후 갱신유효기간', '관능검사원', '배출가스검사원', '책임검사원', '검사 판정시간',
       '자료구분코드', '파일명'],
      dtype='object')

In [282]:
per_d_ins.columns

Index(['정밀검사소명', '정밀검사일자', '정밀검사종류', '정밀검사접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '원동기배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일',
       '정밀검사유효기간', '관능검사판정', '관능검사부적합내용', '정밀검사측정방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값', '산소값', '공기온도', '공기압력',
       '정밀검사판정', '재검사기간', '정밀검사유효기간2', '관능검사등록자', '정밀검사등록자', '정밀검사소대표명',
       '기타내용1', '기타내용2', '기타내용3', '기타내용4', '기타내용5', '원본대장테이블', '정밀검사시간정보',
       '통합접수번호', '데이터생성일자', '수신파일명', '제작등록일자', '데이터구분', '삭제구분', '삭제사유1',
       '삭제사유2', '삭제사유3', '삭제사유4'],
      dtype='object')

### 제원정보와 정기/정밀 매칭 안되는 샘플

In [269]:
df4['차대번호'].isnull().sum()

0

In [263]:
len(df4['차대번호'].unique())

1528557

In [262]:
len(per_ins['차대번호'].unique())

10328354

In [264]:
len(per_d_ins['차대번호'].unique())

14203268

In [265]:
len(set(df4['차대번호']))

1528557

In [266]:
len(set(df4['차대번호']) - set(per_ins['차대번호']))

1047521

In [267]:
len(set(df4['차대번호']) - set(per_d_ins['차대번호']))

540567

In [268]:
len( set(df4['차대번호']) - set(per_ins['차대번호']) - set(per_d_ins['차대번호']) )

323516

In [283]:
df6.shape

(1528557, 65)

In [417]:
gas_col1 = ['차대번호', '검사방법', 
            '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', 
            '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
            '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
            '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
            '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5',
            '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6']
gas_col2 = ['차대번호', '정밀검사측정방법', 
            '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', 
            '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', 
            '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', 
            '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', 
            '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', 
            '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6']

In [418]:
# 53.2s
tdf1 = df6.merge(per_ins[gas_col1], on='차대번호', how='left')
tdf1.shape

(2232455, 84)

In [419]:
# 1m 5.6s
tdf2 = tdf1.merge(per_d_ins[gas_col2], on='차대번호', how='left')
tdf2.shape

(3739980, 103)

In [658]:
# 2m 38.2s
tdf2.to_csv('data/[통합]제원정보_정기_정밀20190701_20220630.csv')

# [로드] [통합]제원정보_정기_정밀20190701_20220630.csv

In [87]:
tdf2 = pd.read_csv('data/[통합]제원정보_정기_정밀20190701_20220630.csv', index_col=0, usecols=['차대번호', '배출가스인증번호', '검사방법', 
           '무부하매연측정치1_x', '무부하매연허용치1_x', '무부하매연판정1_x',
           '무부하매연측정치2_x', '무부하매연허용치2_x', '무부하매연판정2_x',
           '무부하매연측정치3_x', '무부하매연허용치3_x', '무부하매연판정3_x', 
           '무부하매연측정치4_x', '무부하매연허용치4_x', '무부하매연판정4_x', 
           '무부하매연측정치5_x', '무부하매연허용치5_x', '무부하매연판정5_x',
           '무부하매연측정치6_x', '무부하매연허용치6_x', '무부하매연판정6_x',
           '정밀검사측정방법',
           '무부하매연측정치1_y', '무부하매연허용치1_y', '무부하매연판정1_y', 
           '무부하매연측정치2_y', '무부하매연허용치2_y', '무부하매연판정2_y', 
           '무부하매연측정치3_y', '무부하매연허용치3_y', '무부하매연판정3_y',
           '무부하매연측정치4_y', '무부하매연허용치4_y', '무부하매연판정4_y',
           '무부하매연측정치5_y', '무부하매연허용치5_y', '무부하매연판정5_y', 
           '무부하매연측정치6_y', '무부하매연허용치6_y', '무부하매연판정6_y',
           '배출가스인증번호_코드'
           ])

C:\Users\kbjung\AppData\Local\Temp\ipykernel_15180\937433460.py:1: DtypeWarning: Columns (72,75,78,81,84,103) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf2 = pd.read_csv('data/[통합]제원정보_정기_정밀20190701_20220630.csv', index_col=0, usecols=['차대번호', '배출가스인증번호', '검사방법',


In [88]:
tdf2.shape

(3739980, 40)

In [15]:
tdf2.columns

Index(['배출가스인증번호', '검사방법', '무부하매연측정치1_x', '무부하매연허용치1_x', '무부하매연판정1_x',
       '무부하매연측정치2_x', '무부하매연허용치2_x', '무부하매연판정2_x', '무부하매연측정치3_x',
       '무부하매연허용치3_x', '무부하매연판정3_x', '무부하매연측정치4_x', '무부하매연허용치4_x', '무부하매연판정4_x',
       '무부하매연측정치5_x', '무부하매연허용치5_x', '무부하매연판정5_x', '무부하매연측정치6_x',
       '무부하매연허용치6_x', '무부하매연판정6_x', '정밀검사측정방법', '무부하매연측정치1_y', '무부하매연허용치1_y',
       '무부하매연판정1_y', '무부하매연측정치2_y', '무부하매연허용치2_y', '무부하매연판정2_y', '무부하매연측정치3_y',
       '무부하매연허용치3_y', '무부하매연판정3_y', '무부하매연측정치4_y', '무부하매연허용치4_y', '무부하매연판정4_y',
       '무부하매연측정치5_y', '무부하매연허용치5_y', '무부하매연판정5_y', '무부하매연측정치6_y',
       '무부하매연허용치6_y', '무부하매연판정6_y'],
      dtype='object')

In [5]:
tdf2.columns[:50]

Index(['Unnamed: 0', '차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '차량말소YN', 'code1', '시도', '시구군', '폐지여부', '대기관리권역', '대기관리권역YN',
       '총중량_분류', '총중량_코드', '배기량_분류'],
      dtype='object')

In [6]:
tdf2.columns[50:100]

Index(['배기량_코드', '배출가스인증번호_코드', 'wmi', 'WMI', 'MF CODE', 'Identifier',
       'Manufacturer', 'Country', 'Vehicle Types', 'Description', '참고',
       'VIN_length', 'IOQ', 'alpha', '차대번호_10자리', '차대번호_10자리_기준연식', '검사방법',
       '무부하매연측정치1_x', '무부하매연허용치1_x', '무부하매연판정1_x', '무부하매연측정치2_x',
       '무부하매연허용치2_x', '무부하매연판정2_x', '무부하매연측정치3_x', '무부하매연허용치3_x', '무부하매연판정3_x',
       '무부하매연측정치4_x', '무부하매연허용치4_x', '무부하매연판정4_x', '무부하매연측정치5_x',
       '무부하매연허용치5_x', '무부하매연판정5_x', '무부하매연측정치6_x', '무부하매연허용치6_x', '무부하매연판정6_x',
       '정밀검사측정방법', '무부하매연측정치1_y', '무부하매연허용치1_y', '무부하매연판정1_y', '무부하매연측정치2_y',
       '무부하매연허용치2_y', '무부하매연판정2_y', '무부하매연측정치3_y', '무부하매연허용치3_y', '무부하매연판정3_y',
       '무부하매연측정치4_y', '무부하매연허용치4_y', '무부하매연판정4_y', '무부하매연측정치5_y',
       '무부하매연허용치5_y'],
      dtype='object')

In [7]:
tdf2.columns[100:]

Index(['무부하매연판정5_y', '무부하매연측정치6_y', '무부하매연허용치6_y', '무부하매연판정6_y'], dtype='object')

In [8]:
# 8     7MY-HD-14-68
# 10    7MY-HD-24-31
# 16    7MY-HD-14-68
# 17    7MY-HD-14-68
# 18    7MY-HD-14-68

### 정기검사 값 계산

In [89]:
tdf2['검사방법'].unique()

array([nan, '무부하검사(급가속)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [90]:
gas_col = ['배출가스인증번호', '검사방법', 
           '무부하매연측정치1_x', '무부하매연허용치1_x', '무부하매연판정1_x',
           '무부하매연측정치2_x', '무부하매연허용치2_x', '무부하매연판정2_x',
           '무부하매연측정치3_x', '무부하매연허용치3_x', '무부하매연판정3_x', 
           '무부하매연측정치4_x', '무부하매연허용치4_x', '무부하매연판정4_x', 
           '무부하매연측정치5_x', '무부하매연허용치5_x', '무부하매연판정5_x',
           '무부하매연측정치6_x', '무부하매연허용치6_x', '무부하매연판정6_x']
gas_per_ins = tdf2[gas_col].copy()
gas_per_ins.shape

(3739980, 20)

In [91]:
uni = gas_per_ins['배출가스인증번호'].dropna().unique()
uni

array(['7MY-HD-14-68', '7MY-HD-24-31', '7MY-HD-24-09', '7MY-HD-24-28',
       '7MY-KM-14-65', '6MY-SS-0M-24', '7MY-HD-13-58', '7MY-KM-14-63',
       '7MY-HD-14-47', '7MY-HD-14-46', '7MY-DB-24-42', '8MY-KM-13-19',
       '7MY-HD-14-57', '7MY-KM-14-64', '9MY-KM-14-41', '7MY-HD-24-30',
       'FMY-VTK-14-1', '5MY-HD-24-59', '6MY-HD-23-55', '6MY-KM-14-61',
       '7MY-DW-13-07', '7MY-HD-24-29', '5MY-AD-14-18', 'FMY-VTK-14-4',
       '6MY-HD-24-12', '7MY-SY-14-48', '6MY-HD-24-100', '7MY-HD-14-45',
       'FMY-VTK-14-2', 'FMY-VTK-14-3', 'RMY-HD-20', 'VMY-HD-30',
       'WMY-HD-32', '5MY-AD-14-12', '9MY-HD-14-21', '5MY-AD-14-08',
       '7MY-SY-14-60', '7MY-SM-14-36', '6MY-HD-14-97', '7MY-HD-24-08',
       '6MY-KM-24-81', 'CMY-VTK-14-01', 'CMY-VTK-14-02', 'KMY-HD-03',
       '6MY-HD-14-64', 'MMY-HD-11', '5MY-SY-23-58', '6MY-HD-14-63',
       'SMY-KM-32', '6MY-KM-24-06', '7MY-KM-14-52', '8MY-HT-0M-24',
       '6MY-HD-24-44', '7MY-SY-14-34', '5MY-HB-24-02', 'BMY-PPE-14-29',
       '6MY-DW-14-18

In [92]:
len(uni)

300

In [93]:
cal_col = ['무부하매연측정치1_x', '무부하매연측정치2_x', '무부하매연측정치3_x', '무부하매연측정치4_x', '무부하매연측정치5_x', '무부하매연측정치6_x']

In [94]:
gas_per_ins[cal_col].isnull().sum()

무부하매연측정치1_x    2109447
무부하매연측정치2_x    3718889
무부하매연측정치3_x    3718889
무부하매연측정치4_x    3721882
무부하매연측정치5_x    3721882
무부하매연측정치6_x    3721882
dtype: int64

In [95]:
a = [1,2,3,4,5,6]
a[-3:]

[4, 5, 6]

In [96]:
one = 'RMY-HD-20'
temp = gas_per_ins.loc[gas_per_ins['배출가스인증번호'] == one, cal_col].reset_index(drop=True)

In [97]:
temp.head()

,무부하매연측정치1_x,무부하매연측정치2_x,무부하매연측정치3_x,무부하매연측정치4_x,무부하매연측정치5_x,무부하매연측정치6_x
0,0.6,166.0,1.02,0.5,101.0,1.01
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
temp.tail()

,무부하매연측정치1_x,무부하매연측정치2_x,무부하매연측정치3_x,무부하매연측정치4_x,무부하매연측정치5_x,무부하매연측정치6_x
20903,NaN,NaN,NaN,NaN,NaN,NaN
20904,0.0,0.0,0.0,NaN,NaN,NaN
20905,NaN,NaN,NaN,NaN,NaN,NaN
20906,NaN,NaN,NaN,NaN,NaN,NaN
20907,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
temp.mean(axis=1).head()

0    45.021667
1          NaN
2          NaN
3          NaN
4          NaN
dtype: float64

In [100]:
temp = temp.dropna(subset=['무부하매연측정치1_x']).reset_index(drop=True)
temp.head()

,무부하매연측정치1_x,무부하매연측정치2_x,무부하매연측정치3_x,무부하매연측정치4_x,무부하매연측정치5_x,무부하매연측정치6_x
0,0.60,166.0,1.02,0.50,101.0,1.01
1,0.10,114.0,1.08,0.28,166.0,1.01
2,0.30,207.0,1.07,0.23,77.0,1.03
3,0.09,119.0,1.32,0.21,81.0,1.05
4,0.00,93.0,1.03,0.04,57.0,1.00


In [101]:
temp.shape

(2152, 6)

In [102]:
temp.tail()

,무부하매연측정치1_x,무부하매연측정치2_x,무부하매연측정치3_x,무부하매연측정치4_x,무부하매연측정치5_x,무부하매연측정치6_x
2147,0.10,77.0,1.00,0.1,19.0,1.00
2148,0.60,210.0,1.03,0.5,128.0,1.01
2149,0.54,170.0,1.02,0.5,80.0,1.02
2150,0.00,1.0,1.04,0.1,5.0,1.01
2151,0.00,0.0,0.00,NaN,NaN,NaN


In [103]:
temp.iloc[0].to_list()

[0.6, 166.0, 1.02, 0.5, 101.0, 1.01]

In [104]:
temp.iloc[2151].to_list()

[0.0, 0.0, 0.0, nan, nan, nan]

In [105]:
[x for x in temp.iloc[2151].to_list() if np.isnan(x) == False]

[0.0, 0.0, 0.0]

In [106]:
np.mean([x for x in temp.iloc[2151].to_list() if np.isnan(x) == False])

0.0

In [107]:
# 6번 제거하면 안됨
# 다른 방식 사용해야 함
temp = temp.dropna(axis=1)
temp

,무부하매연측정치1_x,무부하매연측정치2_x,무부하매연측정치3_x
0,0.60,166.0,1.02
1,0.10,114.0,1.08
2,0.30,207.0,1.07
3,0.09,119.0,1.32
4,0.00,93.0,1.03
...,...,...,...
2147,0.10,77.0,1.00
2148,0.60,210.0,1.03
2149,0.54,170.0,1.02
2150,0.00,1.0,1.04


In [108]:
gas_yn_col = ['무부하매연판정1_x', '무부하매연판정2_x', '무부하매연판정3_x', '무부하매연판정4_x', '무부하매연판정5_x', '무부하매연판정6_x']

In [109]:
one = 'RMY-HD-20'
temp2 = gas_per_ins.loc[temp.index, gas_yn_col]

KeyError: "None of [Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,\n            ...\n            2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151],\n           dtype='int64', name='차대번호', length=2152)] are in the [index]"

In [110]:
temp2

NameError: name 'temp2' is not defined

In [39]:
temp2.iloc[:, 5].value_counts(normalize=True)

NameError: name 'temp2' is not defined

In [32]:
temp2.iloc[:, 5].value_counts(normalize=True).shape

(1,)

In [33]:
len(temp2.iloc[:, 5].value_counts(normalize=True))

1

In [34]:
uni = gas_per_ins['배출가스인증번호'].dropna().unique()

In [ ]:
# # 정기검사(22분 이상 소요 중, 중단)
# gas_no_list = []
# gas_mean_list = []
# gas_total_no = []
# for one in uni:
#     gas_value_list = []
#     temp = gas_per_ins.loc[gas_per_ins['배출가스인증번호'] == one, cal_col]
#     temp = temp.dropna(subset=['무부하매연측정치1_x']).reset_index(drop=True)
#     temp2 = gas_per_ins.loc[temp.dropna(subset=['무부하매연측정치1_x']).index, gas_yn_col]
#     if temp.shape[0] != 0:
#         for i in range(temp.shape[0]):
#             list1 = [x for x in temp.iloc[i].to_list() if np.isnan(x) == False]
#             if len(list1) == 1:
#                 v = list1[0]
#                 vc = temp2.iloc[:, 0].value_counts(normalize=True)
#                 if vc.shape[0] == 1:
#                     yn = 0
#                 elif vc.shape[0] == 0:
#                     yn = -99
#                 else:
#                     yn = vc[-1]
#             elif len(list1) == 0:
#                 v = 0
#                 yn = -99
#             else:
#                 v = np.mean(list1[-3:])
#                 vc = temp2.iloc[:, 5].value_counts(normalize=True)
#                 if vc.shape[0] == 1:
#                     yn = 0
#                 elif vc.shape[0] == 0:
#                     yn = -99
#                 else:
#                     yn = vc[-1]
#             gas_value_list.append(v)
#         m = np.mean(gas_value_list)
#         n_no = 100 - yn
#         gas_no = 100 - m*4
#         total = gas_no*0.7 + n_no*0.3
#     else:
#         m = -99
#         total = -99

#     gas_mean_list.append(m)
#     gas_no_list.append(one)
#     gas_total_no.append(total)

In [ ]:
# len(gas_mean_list), len(gas_no_list), len(gas_total_no)

In [ ]:
# gas_total_df = pd.DataFrame({'배출가스인증번호':gas_no_list, 'total':gas_total_no})
# gas_total_df.head()

In [308]:
# gas_col = ['배출가스인증번호', '검사방법', '무부하매연측정치1_x', '무부하매연허용치1_x', '무부하매연판정1_x', '정밀검사측정방법', '무부하매연측정치1_y', '무부하매연허용치1_y', '무부하매연판정1_y']
# one = '7MY-HD-14-68'
# temp = tdf2.loc[tdf2['배출가스인증번호'] == one, gas_col]

In [309]:
# m = temp['무부하매연측정치1_x'].mean()
# gas_no = 100 - m * 4

In [319]:
# m, gas_no

(8.827202050798407, 64.69119179680638)

In [313]:
# temp['무부하매연판정1_x'].value_counts(normalize=True)

Y    0.956003
N    0.043997
Name: 무부하매연판정1_x, dtype: float64

In [326]:
# temp['무부하매연판정1_x'].value_counts(normalize=True)

Y    1.0
Name: 무부하매연판정1_x, dtype: float64

In [325]:
# len(temp['무부하매연판정1_x'].value_counts(normalize=True))

1

In [323]:
# temp['무부하매연판정1_x'].value_counts(normalize=True)[-1]

1.0

In [315]:
# yn = temp['무부하매연판정1_x'].value_counts(normalize=True)[1]

In [316]:
# n_no = 100 - yn
# total = gas_no*0.7 + n_no*0.3

In [317]:
# dic = {'배출가스인증번호':one, 'total':total}
# dic

{'배출가스인증번호': '7MY-HD-14-68', 'total': 75.2706351401033}

In [111]:
gas_col = ['배출가스인증번호', '검사방법', '무부하매연측정치1_x', '무부하매연허용치1_x', '무부하매연판정1_x', '정밀검사측정방법', '무부하매연측정치1_y', '무부하매연허용치1_y', '무부하매연판정1_y']

In [112]:
uni = gas_per_ins['배출가스인증번호'].dropna().unique()

In [113]:
# 정밀검사
# 약 1m 12s
gas_no_list = []
gas_total_no = []
for one in uni:
    temp = tdf2.loc[tdf2['배출가스인증번호'] == one, gas_col]
    if temp['무부하매연측정치1_x'].isnull().sum() != 0:
        m = temp['무부하매연측정치1_x'].mean()
        gas_no = 100 - m * 4
    else:
        m = -99
        gas_no = -99
    if temp['무부하매연판정1_x'].isnull().sum() != 0:
        if len(temp['무부하매연판정1_x'].value_counts(normalize=True)) != 1:
            try:
                yn = temp['무부하매연판정1_x'].value_counts(normalize=True)[1]
                n_no = 100 - yn
                total = gas_no*0.7 + n_no*0.3
            except:
                total = -99
        else:
            n_no = 100
            total = gas_no*0.7 + n_no*0.3     
    else:
        total = -99

    gas_no_list.append(one)
    gas_total_no.append(total)

In [114]:
len(gas_no_list), len(gas_total_no)

(300, 300)

In [115]:
gas_total_df = pd.DataFrame({'배출가스인증번호':gas_no_list, 'total':gas_total_no})
gas_total_df.head()

,배출가스인증번호,total
0,7MY-HD-14-68,75.270635
1,7MY-HD-24-31,94.255826
2,7MY-HD-24-09,70.937156
3,7MY-HD-24-28,85.341147
4,7MY-KM-14-65,68.027911


In [116]:
gas_total_df.tail()

,배출가스인증번호,total
295,KMY-KM-05,-99.00
296,KMY-KM-02,100.00
297,KMY-HD-05,-99.00
298,MMY-KM-10,-99.00
299,TY88-배K-09,98.32


In [117]:
gas_total_df[gas_total_df['total'] == -99]

,배출가스인증번호,total
41,CMY-VTK-14-01,-99.0
55,BMY-PPE-14-29,-99.0
70,AMY-PPE-14-654,-99.0
82,AMY-PPE-14-888,-99.0
86,7MY-KM-24-33,-99.0
...,...,...
293,PMY-KM-16,-99.0
294,KMY-KM-04,-99.0
295,KMY-KM-05,-99.0
297,KMY-HD-05,-99.0


In [118]:
gas_total_df[gas_total_df['total'] == -99].shape

(61, 2)

In [119]:
gas_col

['배출가스인증번호',
 '검사방법',
 '무부하매연측정치1_x',
 '무부하매연허용치1_x',
 '무부하매연판정1_x',
 '정밀검사측정방법',
 '무부하매연측정치1_y',
 '무부하매연허용치1_y',
 '무부하매연판정1_y']

In [120]:
# 정밀검사
# 약 1m 14s
gas_no_list2 = []
gas_total_no2 = []
for one in uni:
    temp = tdf2.loc[tdf2['배출가스인증번호'] == one, gas_col]
    if temp['무부하매연측정치1_y'].isnull().sum() != 0:
        m = temp['무부하매연측정치1_y'].mean()
        gas_no = 100 - m * 4
    else:
        m = -99
        gas_no = -99
    if temp['무부하매연판정1_y'].isnull().sum() != 0:
        if len(temp['무부하매연판정1_y'].value_counts(normalize=True)) != 1:
            try:
                yn = temp['무부하매연판정1_y'].value_counts(normalize=True)[1]
                n_no = 100 - yn
                total = gas_no*0.7 + n_no*0.3
            except:
                total = -99
        else:
            n_no = 100
            total = gas_no*0.7 + n_no*0.3     
    else:
        total = -99

    gas_no_list2.append(one)
    gas_total_no2.append(total)

In [121]:
len(gas_no_list2), len(gas_total_no2)

(300, 300)

In [122]:
gas_total_df2 = pd.DataFrame({'배출가스인증번호':gas_no_list2, 'total':gas_total_no2})
gas_total_df2.head()

,배출가스인증번호,total
0,7MY-HD-14-68,74.004640
1,7MY-HD-24-31,80.843103
2,7MY-HD-24-09,59.467949
3,7MY-HD-24-28,71.745346
4,7MY-KM-14-65,65.183820


In [123]:
gas_total_df2.tail()

,배출가스인증번호,total
295,KMY-KM-05,-99.000000
296,KMY-KM-02,99.552000
297,KMY-HD-05,-99.000000
298,MMY-KM-10,-99.000000
299,TY88-배K-09,98.310667


In [124]:
gas_total = gas_total_df.merge(gas_total_df2, on='배출가스인증번호')
gas_total.head()

,배출가스인증번호,total_x,total_y
0,7MY-HD-14-68,75.270635,74.004640
1,7MY-HD-24-31,94.255826,80.843103
2,7MY-HD-24-09,70.937156,59.467949
3,7MY-HD-24-28,85.341147,71.745346
4,7MY-KM-14-65,68.027911,65.183820


In [125]:
gas_total.loc[gas_total['total_x'] == -99, 'total_x'] = np.nan
gas_total.loc[gas_total['total_y'] == -99, 'total_y'] = np.nan

In [126]:
gas_total['total_diff'] = gas_total['total_x'] - gas_total['total_y']
gas_total.head()

,배출가스인증번호,total_x,total_y,total_diff
0,7MY-HD-14-68,75.270635,74.004640,1.265995
1,7MY-HD-24-31,94.255826,80.843103,13.412722
2,7MY-HD-24-09,70.937156,59.467949,11.469207
3,7MY-HD-24-28,85.341147,71.745346,13.595802
4,7MY-KM-14-65,68.027911,65.183820,2.844091


In [127]:
gas_total.loc[gas_total['total_diff'] != 0][:30]

,배출가스인증번호,total_x,total_y,total_diff
0,7MY-HD-14-68,75.270635,74.004640,1.265995
1,7MY-HD-24-31,94.255826,80.843103,13.412722
2,7MY-HD-24-09,70.937156,59.467949,11.469207
3,7MY-HD-24-28,85.341147,71.745346,13.595802
4,7MY-KM-14-65,68.027911,65.183820,2.844091
5,6MY-SS-0M-24,80.138496,75.174561,4.963934
6,7MY-HD-13-58,91.636014,93.580258,-1.944244
7,7MY-KM-14-63,70.290213,69.139125,1.151088
8,7MY-HD-14-47,68.069629,72.908868,-4.839239
9,7MY-HD-14-46,72.200863,72.841197,-0.640334


In [128]:
gas_total.loc[gas_total['total_diff'] != 0][30:]

,배출가스인증번호,total_x,total_y,total_diff
30,RMY-HD-20,99.047771,99.203125,-0.155353
31,VMY-HD-30,98.705196,99.296536,-0.591340
32,WMY-HD-32,99.404678,99.309852,0.094826
33,5MY-AD-14-12,87.386364,98.320000,-10.933636
34,9MY-HD-14-21,72.667981,67.736653,4.931328
...,...,...,...,...
295,KMY-KM-05,NaN,NaN,NaN
296,KMY-KM-02,100.000000,99.552000,0.448000
297,KMY-HD-05,NaN,NaN,NaN
298,MMY-KM-10,NaN,NaN,NaN


In [129]:
gas_total.isnull().sum()

배출가스인증번호       0
total_x       61
total_y       55
total_diff    72
dtype: int64

In [130]:
gas_total = gas_total.fillna(-1)

In [131]:
gas_total.loc[gas_total['total_x'] == -1, 'total_x'].index

Int64Index([ 41,  55,  70,  82,  86,  87,  88,  89,  91,  98,  99, 101, 102,
            103, 104, 106, 107, 113, 117, 136, 151, 165, 175, 178, 179, 184,
            190, 191, 193, 195, 199, 200, 202, 203, 204, 205, 207, 208, 209,
            211, 212, 213, 214, 215, 216, 217, 219, 220, 240, 263, 273, 276,
            281, 284, 287, 289, 293, 294, 295, 297, 298],
           dtype='int64')

In [132]:
gas_total.loc[gas_total['total_y'] == -1, 'total_y'].index

Int64Index([ 41,  70,  82,  87,  88,  89,  91,  98,  99, 101, 102, 103, 104,
            106, 107, 108, 113, 114, 117, 136, 142, 143, 175, 184, 187, 188,
            190, 191, 194, 195, 200, 201, 202, 203, 204, 205, 209, 212, 213,
            214, 216, 219, 220, 238, 240, 259, 263, 276, 277, 284, 293, 294,
            295, 297, 298],
           dtype='int64')

In [133]:
set(gas_total.loc[gas_total['total_x'] == -1, 'total_x'].index) == set(gas_total.loc[gas_total['total_y'] == -1, 'total_y'].index)

False

In [134]:
gas_total.columns

Index(['배출가스인증번호', 'total_x', 'total_y', 'total_diff'], dtype='object')

In [135]:
gas_total.shape

(300, 4)

In [152]:
tdf2.columns

Index(['배출가스인증번호', '배출가스인증번호_코드', '검사방법', '무부하매연측정치1_x', '무부하매연허용치1_x',
       '무부하매연판정1_x', '무부하매연측정치2_x', '무부하매연허용치2_x', '무부하매연판정2_x', '무부하매연측정치3_x',
       '무부하매연허용치3_x', '무부하매연판정3_x', '무부하매연측정치4_x', '무부하매연허용치4_x', '무부하매연판정4_x',
       '무부하매연측정치5_x', '무부하매연허용치5_x', '무부하매연판정5_x', '무부하매연측정치6_x',
       '무부하매연허용치6_x', '무부하매연판정6_x', '정밀검사측정방법', '무부하매연측정치1_y', '무부하매연허용치1_y',
       '무부하매연판정1_y', '무부하매연측정치2_y', '무부하매연허용치2_y', '무부하매연판정2_y', '무부하매연측정치3_y',
       '무부하매연허용치3_y', '무부하매연판정3_y', '무부하매연측정치4_y', '무부하매연허용치4_y', '무부하매연판정4_y',
       '무부하매연측정치5_y', '무부하매연허용치5_y', '무부하매연판정5_y', '무부하매연측정치6_y',
       '무부하매연허용치6_y', '무부하매연판정6_y'],
      dtype='object')

In [136]:
tdf3 = tdf2.drop_duplicates(['배출가스인증번호'], keep='first')
tdf3.shape

(301, 40)

In [153]:
gas_total.columns

Index(['배출가스인증번호', 'total_x', 'total_y', 'total_diff'], dtype='object')

In [137]:
gas_total2 = gas_total.merge(tdf3, on='배출가스인증번호', how='left')

In [138]:
gas_total2.shape

(300, 43)

In [156]:
gas_total2 = gas_total2.drop('배출가스인증번호_코드', axis=1)

In [160]:
gas_total2 = gas_total[['배출가스인증번호', 'total_x', 'total_y', 'total_diff']]

In [161]:
gas_total2.columns

Index(['배출가스인증번호', 'total_x', 'total_y', 'total_diff'], dtype='object')

In [159]:
tdf3.columns

Index(['배출가스인증번호', '배출가스인증번호_코드', '검사방법', '무부하매연측정치1_x', '무부하매연허용치1_x',
       '무부하매연판정1_x', '무부하매연측정치2_x', '무부하매연허용치2_x', '무부하매연판정2_x', '무부하매연측정치3_x',
       '무부하매연허용치3_x', '무부하매연판정3_x', '무부하매연측정치4_x', '무부하매연허용치4_x', '무부하매연판정4_x',
       '무부하매연측정치5_x', '무부하매연허용치5_x', '무부하매연판정5_x', '무부하매연측정치6_x',
       '무부하매연허용치6_x', '무부하매연판정6_x', '정밀검사측정방법', '무부하매연측정치1_y', '무부하매연허용치1_y',
       '무부하매연판정1_y', '무부하매연측정치2_y', '무부하매연허용치2_y', '무부하매연판정2_y', '무부하매연측정치3_y',
       '무부하매연허용치3_y', '무부하매연판정3_y', '무부하매연측정치4_y', '무부하매연허용치4_y', '무부하매연판정4_y',
       '무부하매연측정치5_y', '무부하매연허용치5_y', '무부하매연판정5_y', '무부하매연측정치6_y',
       '무부하매연허용치6_y', '무부하매연판정6_y'],
      dtype='object')

In [163]:
tdf3.shape

(301, 40)

In [164]:
df7 = tdf2.merge(gas_total2, on='배출가스인증번호', how='left')
df7.shape

(3739980, 43)

In [165]:
df7['total_x'].isnull().sum()

139481

In [166]:
df7[df7['total_x'] == -99]

,배출가스인증번호,배출가스인증번호_코드,검사방법,무부하매연측정치1_x,무부하매연허용치1_x,무부하매연판정1_x,무부하매연측정치2_x,무부하매연허용치2_x,무부하매연판정2_x,무부하매연측정치3_x,...,무부하매연판정4_y,무부하매연측정치5_y,무부하매연허용치5_y,무부하매연판정5_y,무부하매연측정치6_y,무부하매연허용치6_y,무부하매연판정6_y,total_x,total_y,total_diff


In [167]:
df7.columns

Index(['배출가스인증번호', '배출가스인증번호_코드', '검사방법', '무부하매연측정치1_x', '무부하매연허용치1_x',
       '무부하매연판정1_x', '무부하매연측정치2_x', '무부하매연허용치2_x', '무부하매연판정2_x', '무부하매연측정치3_x',
       '무부하매연허용치3_x', '무부하매연판정3_x', '무부하매연측정치4_x', '무부하매연허용치4_x', '무부하매연판정4_x',
       '무부하매연측정치5_x', '무부하매연허용치5_x', '무부하매연판정5_x', '무부하매연측정치6_x',
       '무부하매연허용치6_x', '무부하매연판정6_x', '정밀검사측정방법', '무부하매연측정치1_y', '무부하매연허용치1_y',
       '무부하매연판정1_y', '무부하매연측정치2_y', '무부하매연허용치2_y', '무부하매연판정2_y', '무부하매연측정치3_y',
       '무부하매연허용치3_y', '무부하매연판정3_y', '무부하매연측정치4_y', '무부하매연허용치4_y', '무부하매연판정4_y',
       '무부하매연측정치5_y', '무부하매연허용치5_y', '무부하매연판정5_y', '무부하매연측정치6_y',
       '무부하매연허용치6_y', '무부하매연판정6_y', 'total_x', 'total_y', 'total_diff'],
      dtype='object')

In [168]:
df7['total_x'] = df7['total_x'].fillna(-1)
df7['total_x'].isnull().sum()

0

In [169]:
grade_list = []
for one in tqdm(df7['total_x']):
    if 100 >= one > 75:
        grade_list.append('A')
    elif 75 >= one > 50:
        grade_list.append('B')
    elif 50 >= one > 25:
        grade_list.append('C')
    elif 25 >= one >= 0:
        grade_list.append('D')
    elif one == -1:
        grade_list.append('miss')

100%|██████████| 3739980/3739980 [00:02<00:00, 1586294.40it/s]


In [170]:
len(grade_list)

3739980

In [171]:
df7['Grade_x'] = grade_list

In [172]:
df7['배출가스인증번호_코드'].unique()

array([nan, 'EN088', 'EN095', 'EN091', 'EN092', 'EN110', 'EN056', 'EN083',
       'EN108', 'EN086', 'EN085', 'EN066', 'EN172', 'EN087', 'EN109',
       'EN195', 'EN094', 'EN221', 'EN018', 'EN042', 'EN049', 'EN078',
       'EN093', 'EN010', 'EN224', 'EN044', 'EN133', 'EN043', 'EN084',
       'EN222', 'EN223', 'EN257', 'EN283', 'EN289', 'EN009', 'EN193',
       'EN008', 'EN135', 'EN128', 'EN041', 'EN090', 'EN051', 'EN219',
       'EN220', 'EN228', 'EN039', 'EN240', 'EN028', 'EN038', 'EN267',
       'EN050', 'EN107', 'EN167', 'EN045', 'EN132', 'EN014', 'EN217',
       'EN036', 'EN060', 'EN180', 'EN131', 'EN046', 'EN020', 'EN151',
       'EN071', 'EN296', 'EN111', 'EN204', 'EN192', 'EN031', 'EN126',
       'EN208', 'EN152', 'EN074', 'EN070', 'EN069', 'EN072', 'EN116',
       'EN150', 'EN295', 'EN187', 'EN203', 'EN174', 'EN212', 'EN017',
       'EN040', 'EN253', 'EN112', 'EN215', 'EN216', 'EN211', 'EN117',
       'EN213', 'EN138', 'EN173', 'EN139', 'EN153', 'EN055', 'EN196',
       'EN210',

In [173]:
df7.reset_index().columns

Index(['index', '배출가스인증번호', '배출가스인증번호_코드', '검사방법', '무부하매연측정치1_x',
       '무부하매연허용치1_x', '무부하매연판정1_x', '무부하매연측정치2_x', '무부하매연허용치2_x', '무부하매연판정2_x',
       '무부하매연측정치3_x', '무부하매연허용치3_x', '무부하매연판정3_x', '무부하매연측정치4_x',
       '무부하매연허용치4_x', '무부하매연판정4_x', '무부하매연측정치5_x', '무부하매연허용치5_x', '무부하매연판정5_x',
       '무부하매연측정치6_x', '무부하매연허용치6_x', '무부하매연판정6_x', '정밀검사측정방법', '무부하매연측정치1_y',
       '무부하매연허용치1_y', '무부하매연판정1_y', '무부하매연측정치2_y', '무부하매연허용치2_y', '무부하매연판정2_y',
       '무부하매연측정치3_y', '무부하매연허용치3_y', '무부하매연판정3_y', '무부하매연측정치4_y',
       '무부하매연허용치4_y', '무부하매연판정4_y', '무부하매연측정치5_y', '무부하매연허용치5_y', '무부하매연판정5_y',
       '무부하매연측정치6_y', '무부하매연허용치6_y', '무부하매연판정6_y', 'total_x', 'total_y',
       'total_diff', 'Grade_x'],
      dtype='object')

In [174]:
df7.reset_index()['index'].head()

0    0
1    1
2    2
3    3
4    4
Name: index, dtype: int64

In [247]:
df7.reset_index().groupby(['Grade_x'])['배출가스인증번호_코드'].count()

Grade_x
A       2080193
B       1500812
C         17882
miss       1612
Name: 배출가스인증번호_코드, dtype: int64

In [248]:
df7.drop_duplicates(['Grade_x', '배출가스인증번호_코드'], keep='first').groupby(['Grade_x'])['배출가스인증번호_코드'].count()

Grade_x
A       191
B        39
C         9
miss     61
Name: 배출가스인증번호_코드, dtype: int64

In [249]:
df7.drop_duplicates(['Grade_x', '배출가스인증번호_코드'], keep='first').groupby(['Grade_x'])['배출가스인증번호_코드'].count().to_excel('analysis/[G4][정기검사][배출가스인증번호]등급별_코드_유니크_통계.xlsx')

In [177]:
df7['Grade_x'].unique()

array(['miss', 'A', 'B', 'C'], dtype=object)

In [178]:
df7.loc[df7['Grade_x'] == 'A'].shape

(2080193, 44)

In [179]:
df7.loc[df7['Grade_x'] == 'B'].shape

(1500812, 44)

In [180]:
df7.loc[df7['Grade_x'] == 'C'].shape

(17882, 44)

In [181]:
df7.loc[df7['Grade_x'] == 'D'].shape

(0, 44)

In [182]:
df7.loc[df7['Grade_x'] == 'miss'].shape

(141093, 44)

In [183]:
df7['total_y'].isnull().sum()

139481

In [184]:
df7['total_y'] = df7['total_y'].fillna(-1)
df7['total_y'].isnull().sum()

0

In [185]:
grade_list2 = []
for one in tqdm(df7['total_y']):
    if 100 >= one > 75:
        grade_list2.append('A')
    elif 75 >= one > 50:
        grade_list2.append('B')
    elif 50 >= one > 25:
        grade_list2.append('C')
    elif 25 >= one >= 0:
        grade_list2.append('D')
    elif one == -1:
        grade_list2.append('miss')

100%|██████████| 3739980/3739980 [00:02<00:00, 1526740.97it/s]


In [186]:
len(grade_list2)

3739980

In [187]:
df7['Grade_y'] = grade_list2

In [202]:
df7.reset_index().groupby(['Grade_y', '배출가스인증번호_코드'])['배출가스인증번호_코드'].count()

Grade_y  배출가스인증번호_코드
A        EN001          1079
         EN005           500
         EN006           178
         EN007            69
         EN008          1546
                        ... 
miss     EN249            76
         EN261            49
         EN280           200
         EN281           386
         EN293             4
Name: 배출가스인증번호_코드, Length: 300, dtype: int64

In [203]:
df7.reset_index().groupby(['Grade_y', '배출가스인증번호_코드'])['배출가스인증번호_코드'].count().to_excel('analysis/[G4][정밀검사][배출가스인증번호]등급별_배인코드별_통계.xlsx')

In [250]:
df7.drop_duplicates(['Grade_y', '배출가스인증번호_코드'], keep='first').groupby(['Grade_y'])['배출가스인증번호_코드'].count()

Grade_y
A       169
B        62
C        14
miss     55
Name: 배출가스인증번호_코드, dtype: int64

In [251]:
df7.drop_duplicates(['Grade_y', '배출가스인증번호_코드'], keep='first').groupby(['Grade_y'])['배출가스인증번호_코드'].count().to_excel('analysis/[G4][정밀검사][배출가스인증번호]등급별_배인코드_유니크_통계.xlsx')

In [191]:
df7[df7['Grade_y'] == 'A'].shape

(707470, 45)

In [192]:
df7[df7['Grade_y'] == 'B'].shape

(2832719, 45)

In [193]:
df7[df7['Grade_y'] == 'C'].shape

(59236, 45)

In [194]:
df7[df7['Grade_y'] == 'D'].shape

(0, 45)

In [195]:
df7[df7['Grade_y'] == 'miss'].shape

(140555, 45)

# 코드 마지막